# Make figures

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
import numpy as np
import os
import sys
import seaborn as sns
from tqdm.auto import tqdm
from sklearn.linear_model import LinearRegression
from matplotlib.patches import Rectangle
import matplotlib
import rioxarray as rxr
import xarray as xr
import warnings
warnings.filterwarnings('ignore')

In [ ]:
base_path = '/Users/raineyaberle/Research/PhD/snow_cover_mapping/snow-cover-mapping-application/'
sys.path.append(os.path.join(base_path, 'functions'))
import model_analyze_utils as f

scm_path = '/Volumes/LaCie/raineyaberle/Research/PhD/snow_cover_mapping/'
# scm_path = '/Users/raineyaberle/Research/PhD/snow_cover_mapping/'
figures_out_path = os.path.join(base_path, 'figures')
eras_fn = os.path.join(scm_path, 'analysis', 'all_era_data.csv')
clusters_fn = os.path.join(scm_path, 'analysis', 'climate_clusters.csv')
aois_fn = os.path.join(scm_path, 'analysis', 'all_aois.shp')

## Define some colormaps

In [ ]:
# Climate clusters
cluster_cmap_dict = {'W. Aleutians': '#dd3497', 
                     'Continental': '#a6611a',
                     'Transitional-Continental': '#dfc27d',
                     'Transitional-Maritime': '#80cdc1',
                     'Maritime': '#018571'}

cmap = plt.cm.Greys
n = 10
subregions_cmap_dict_grey = {'Alaska Range': cmap(0/n),
                             'Aleutians': cmap(1/n),
                             'W. Chugach Mtns.': cmap(2/n),
                             'St. Elias Mtns.': cmap(3/n),
                             'N. Coast Ranges': cmap(4/n),
                             'N. Rockies': cmap(5/n),
                             'N. Cascades': cmap(6/n),
                             'C. Rockies': cmap(7/n),
                             'S. Cascades': cmap(8/n),
                             'S. Rockies': cmap(9/n)}

## Define order of clusters and subregions for plotting

In [ ]:
cluster_order = ['W. Aleutians', 'Maritime', 'Transitional-Maritime', 'Transitional-Continental', 'Continental']
subregion_order = ['N. Rockies', 'Alaska Range', 'W. Chugach Mtns.', 'St. Elias Mtns.', 'N. Coast Ranges',
                   'Aleutians', 'N. Cascades', 'C. Rockies', 'S. Cascades', 'S. Rockies']
# separate subregions for plotting
group1 = ['N. Rockies', 'W. Chugach Mtns.', 'N. Coast Ranges', 'N. Cascades', 'S. Cascades']
group2 = ['Alaska Range', 'St. Elias Mtns.', 'Aleutians', 'C. Rockies', 'S. Rockies']

## Figure 1. Median AARs and timing comparisons

In [ ]:
cmap = sns.color_palette('mako', n_colors=len(subregion_order)+1)
cmap_dict = dict([[subregion, color] for subregion, color in list(zip(subregion_order, cmap))])

# -----Load glacier boundaries
aois = gpd.read_file(aois_fn)
print('Glacier boundaries loaded')

# -----Load climate clusters
clusters = pd.read_csv(clusters_fn)

# -----Load median AARs for all sites
min_scs_fn = os.path.join(scm_path, 'analysis', 'min_snow_cover_stats.csv')
min_scs = pd.read_csv(min_scs_fn)
# Add difference from September AAR
# min_scs['AAR_P50_diff'] = min_scs['AAR_P50_WOY39'] - min_scs['AAR_P50_min']
# Add Subregion and climate cluster info
min_scs[['CenLon', 'CenLat', 'Subregion', 'clustName']] = 0, 0, '', ''
for rgi_id in min_scs['RGIId'].drop_duplicates().values:
    cenlon, cenlat, subregion = aois.loc[aois['RGIId']==rgi_id, ['CenLon', 'CenLat', 'Subregion']].values[0]
    cluster = clusters.loc[clusters['RGIId']==rgi_id, 'clustName'].values
    min_scs.loc[min_scs['RGIId']==rgi_id, ['CenLon', 'CenLat', 'Subregion']] = cenlon, cenlat, subregion
# Sort by subregion order
min_scs['Subregion'] = pd.Categorical(min_scs['Subregion'], subregion_order)
min_scs.sort_values(by='Subregion', inplace=True)
print('Median AARs loaded')

# -----Load melt season timings estimate
melt_season_fn = os.path.join(scm_path, 'analysis', 'melt_season_timing.csv')
melt_season = pd.read_csv(melt_season_fn)
# Add subregion column
melt_season['Subregion'] = ''
for rgi_id in melt_season['RGIId'].drop_duplicates().values:
    subregion = aois.loc[aois['RGIId']==rgi_id, 'Subregion'].values[0]
    melt_season.loc[melt_season['RGIId']==rgi_id, 'Subregion'] = subregion
# Sort by subregion order
melt_season['Subregion'] = pd.Categorical(melt_season['Subregion'], subregion_order)
melt_season.sort_values(by='Subregion', inplace=True)
print('Melt season timing loaded')

In [ ]:
fontsize=12
plt.rcParams.update({'font.size': fontsize, 'font.sans-serif':'Arial'})
gs = matplotlib.gridspec.GridSpec(3,10, wspace=0)
fig = plt.figure(figsize=(8,14))
ax = []

# Iterate over subregions
median_color = 'w'
Iax = -1
for i, subregion in enumerate(min_scs['Subregion'].unique()):
    # a) AARs
    ax.append(fig.add_subplot(gs[0,i]))
    Iax += 1
    min_scs_subregion = min_scs.loc[min_scs['Subregion']==subregion]
    sns.boxplot(data=min_scs_subregion, x=i, y='AAR', showfliers=False,
                color=cmap_dict[subregion], medianprops=dict(color='w', linewidth=1.5), ax=ax[Iax])
    ax[Iax].set_ylim(-0.01,1)
    ax[Iax].set_xticks([])
    ax[Iax].spines[['right', 'top']].set_visible(False)
    if i==0:
        ax[Iax].set_ylabel('AAR')
        ax[Iax].text(0.2, 0.9, 'a', transform=ax[Iax].transAxes, 
                     fontweight='bold', fontsize=fontsize+3, color='k')
    else:
        ax[Iax].set_ylabel('')
        ax[Iax].set_yticks([])
        ax[Iax].spines['left'].set_visible(False)
    ax[Iax].set_title(subregion, rotation=90, color=cmap_dict[subregion], fontweight='bold')
    
    # b) AAR timing and melt season duration
    ax.append(fig.add_subplot(gs[1,i]))
    Iax += 1
    k = sns.kdeplot(min_scs_subregion['WOY'], vertical=True, color=cmap_dict[subregion], 
                    fill=True, alpha=1, ax=ax[Iax], zorder=2)
    ax[Iax].set_ylim(13,45)
    median = min_scs_subregion['WOY'].median()
    ax[Iax].plot([0, ax[Iax].get_xlim()[1]*0.9], [median, median], '-', color='w', zorder=3)
    ax[Iax].plot([0,0], [15,45], '-', color=cmap_dict[subregion], linewidth=2)
    melt_season_subregion = melt_season.loc[melt_season['Subregion']==subregion]
    melt_season_start = melt_season_subregion['melt_season_start_WOY'].mean()
    melt_season_end = melt_season_subregion['melt_season_end_WOY'].mean()
    ax[Iax].fill_between([0, ax[Iax].get_xlim()[1]], 
                         [melt_season_start, melt_season_start],  
                         [melt_season_end, melt_season_end],
                         facecolor='k', edgecolor='None', alpha=0.2, zorder=1, label='Melt season duration')
    if i==4:
        ax[Iax].legend(loc='upper center', frameon=False, bbox_to_anchor=[0.87, 0.92, 0.2, 0.2])
    ax[Iax].spines[['left', 'right', 'top', 'bottom']].set_visible(False)
    ax[Iax].set_yticks([18, 22, 26, 31, 35, 39, 44])
    ax[Iax].set_ylabel('')
    ax[Iax].set_xticks([])
    ax[Iax].set_xlabel('')
    if i==0:
        ax[Iax].set_yticklabels(['May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov'])
        ax[Iax].set_ylabel('Snow minimum timing')
        ax[Iax].text(0.2, 0.9, 'b', transform=ax[Iax].transAxes, 
                     fontweight='bold', fontsize=fontsize+3, color='k')
    else:
        ax[Iax].set_yticklabels([])
        
    # c) Observed vs. Sept 1 AAR
    # ax.append(fig.add_subplot(gs[2,i]))
    # Iax += 1
    # min_scs_subregion_melt = pd.melt(
    #     min_scs_subregion,
    #     id_vars=['RGIId', 'Subregion'],
    #     value_vars=['AAR_P50_min', f'AAR_P50_WOY39'],
    #     var_name='AAR_type',
    #     value_name='AAR'
    # )
    # vp = sns.violinplot(data=min_scs_subregion_melt, y='AAR', hue='AAR_type', cut=0, orient='v',
    #                     split=True, inner="quart", fill=True, ax=ax[Iax],
    #                     linewidth=1, dodge=True, width=0.4,
    # )#palette=cmap_dict)
    # violins = vp.collections
    # ax[Iax].set_ylim(-0.01,1)
    # ax[Iax].set_xlabel('')
    # ax[Iax].set_xticks([])
    # ax[Iax].spines[['right', 'top']].set_visible(False)
    # handles, labels = ax[Iax].get_legend_handles_labels()
    # ax[Iax].legend().remove()
    # if i==0:
    #     ax[Iax].text(0.2, 0.9, 'c', transform=ax[Iax].transAxes, 
    #                  fontweight='bold', fontsize=fontsize+3, color='k')
    # else:
    #     ax[Iax].set_yticks([])
    #     ax[Iax].set_ylabel('')
    #     ax[Iax].spines['left'].set_visible(False)
    # if i==4:
    #     labels = ['Observed AAR', 'September 1 SAR']
    #     ax[Iax].legend(handles, labels, loc='upper center', ncols=2, 
    #                    frameon=False, bbox_to_anchor=[0.8, 0.92, 0.2, 0.2])

plt.show()

# Save figure
# fig_fn = os.path.join(figures_out_path, 'fig01_median_aars+timings.png')
# fig.savefig(fig_fn, dpi=300, bbox_inches='tight')
# print('Figure saved to file:', fig_fn)

In [ ]:
# Check WOY and months
# df = pd.DataFrame({'Date': pd.date_range('2013-01-01', '2023-12-30')})
# df['WOY'] = df['Date'].dt.isocalendar().week
# df['year'] = df['Date'].dt.year
# df['month'] = df['Date'].dt.month
# df['day'] = df['Date'].dt.day
# df.loc[(df['month']==10) & (df['day']==1)]['WOY']#.median()

In [ ]:
# Print statistics

print('All sites:')
print('-----------')
print('---AARs---')
print(min_scs['AAR'].describe())
print('\n---WOYs---')
print(min_scs['WOY'].describe()) 

print(' ')
print('By subregion:')
print('-----------')
print('---AARs---')
print(min_scs.groupby('Subregion')['AAR'].describe())

print('\n---WOYs---')
print(min_scs.groupby('Subregion')['WOY'].describe())


In [ ]:
# Look at which sites had AARs < 0.1
aar_lt_p1 = min_scs.loc[min_scs['AAR'] < 0.1]
print(len(aar_lt_p1))
aar_lt_p1

## Figure 2. Melt factors of snow

In [ ]:
# Load AOIs
aois = gpd.read_file(aois_fn)
print('AOIs loaded')

# Load climate clusters / mean climate
clusters = pd.read_csv(clusters_fn)
print('Climate clusters loaded')

# Load melt factors
fsnow_obs_fn = os.path.join(scm_path, 'analysis', 'fsnow_observed.csv')
fsnow_obs = pd.read_csv(fsnow_obs_fn)
# add centroid coordinates and climate cluster
fsnow_obs[['clustName', 'Subregion', 'CenLon', 'CenLat']] = '', '', 0, 0
for rgi_id in fsnow_obs['RGIId'].drop_duplicates().values:
    cluster = clusters.loc[clusters['RGIId']==rgi_id, 'clustName'].values[0]
    cenLon, cenLat, subregion = aois.loc[aois['RGIId']==rgi_id, ['CenLon', 'CenLat', 'Subregion']].values[0]
    fsnow_obs.loc[fsnow_obs['RGIId']==rgi_id, ['clustName', 'Subregion', 'CenLon', 'CenLat']] = cluster, subregion, cenLon, cenLat
print('Melt factors of snow loaded')

# Load RGI O2 Regions
rgi_O2_fn = os.path.join(scm_path, '..', 'GIS_data', 'RGI', 'RGIv7_02Regions', 
                                'RGI2000-v7.0-o2regions-Alaska-westernCanadaUS_clipped_to_country_outlines.shp')
rgi_O2 = gpd.read_file(rgi_O2_fn)
# remove Brooks Range
rgi_O2 = rgi_O2.loc[rgi_O2['o2region']!='01-01']
# add subregion name and color column
rgi_O2[['Subregion', 'color']] = '', ''
for i, o1o2 in enumerate(rgi_O2['o2region'].drop_duplicates().values):
    o1 = int(o1o2[0:2])
    o2 = int(o1o2[3:])
    subregion_name, color = f.determine_subregion_name_color(o1, o2)
    rgi_O2.loc[rgi_O2['o2region']==o1o2, 'Subregion'] = subregion_name
print('RGI O2 regions loaded')

# Load GTOPO30
gtopo_fn = '/Users/raineyaberle/Research/PhD/GIS_data/GTOPO30_clip.tif'
gtopo = rxr.open_rasterio(gtopo_fn)
gtopo = xr.where(gtopo==-32768, np.nan, gtopo)
print('GTOPO30 loaded')


In [ ]:
# -----Plot
fontsize=12
plt.rcParams.update({'font.size':fontsize, 'font.sans-serif':'Arial'})
fig = plt.figure(figsize=(10,14))
gs = matplotlib.gridspec.GridSpec(2, 2, height_ratios=[1.5,1])
ax = [fig.add_subplot(gs[0,:]),
      fig.add_subplot(gs[1,0]),
      fig.add_subplot(gs[1,1])]

### a) Map view
# GTOPO hillshade
ls = matplotlib.colors.LightSource(azdeg=90, altdeg=45)
ax[0].imshow(ls.hillshade(gtopo.data[0], vert_exag=0.002), cmap='gray', alpha=0.5,
             extent=(np.min(gtopo.x.data), np.max(gtopo.x.data), 
                     np.min(gtopo.y.data), np.max(gtopo.y.data)))
# RGI O2 region outlines
color = '#525252'
rgi_O2.plot(ax=ax[0], alpha=1.0, facecolor='None', edgecolor=color, linewidth=1)
ax[0].set_yticks(np.linspace(45, 65, num=6))
ax[0].set_xlim(-167, -112)
ax[0].set_ylim(46, 67)
ax[0].set_xlabel('Longitude')
ax[0].set_ylabel('Latitude')
ax[0].set_aspect(2.1)
# Melt factors
sns.scatterplot(data=fsnow_obs, x='CenLon', y='CenLat', edgecolor='k', linewidth=0.5, 
                hue='clustName', palette=cluster_cmap_dict, alpha=1, size='fsnow_obs', 
                sizes=(5,100), ax=ax[0])
# handles, labels = ax[0].get_legend_handles_labels()
# labels = [x.replace('clustName', 'Climate class') for x in labels]
ax[0].legend().remove()
# fig.legend(handles, labels, loc='center right', bbox_to_anchor=[0.7, 0.38, 0.2, 0.2], fontsize=fontsize-1)
# Add region labels and arrows
fontweight = 'bold'
ax[0].text(-163, 56, 'Aleutians', color=color, rotation=35, fontsize=fontsize-3, fontweight=fontweight)
ax[0].text(-158, 62.3, 'Alaska Range', color=color, rotation=0, fontsize=fontsize-3, fontweight=fontweight)
ax[0].text(-147.9, 57.8, 'W. Chugach \nMtns.', color=color, rotation=0, horizontalalignment='center', fontsize=fontsize-3, fontweight=fontweight)
ax[0].arrow(-147.6, 58.8, 0, 0.8, color=color, linewidth=2, head_width=0.25, head_length=0.2)
ax[0].text(-141.7, 57.7, 'St. Elias \nMtns.', color=color, rotation=0, horizontalalignment='center', fontsize=fontsize-3, fontweight=fontweight)
ax[0].arrow(-141.5, 58.7, 0, 0.8, color=color, linewidth=2, head_width=0.25, head_length=0.2)
ax[0].text(-139.6, 56.4, 'N. Coast \nRanges', color=color, rotation=0, horizontalalignment='center', fontsize=fontsize-3, fontweight=fontweight)
ax[0].arrow(-137.3, 56.8, 1.3, 0, color=color, linewidth=2, head_width=0.25, head_length=0.2)
ax[0].text(-133, 51.3, 'N. Cascades', color=color, rotation=0, horizontalalignment='center', fontsize=fontsize-3, fontweight=fontweight)
ax[0].arrow(-129.4, 51.4, 1.3, 0, color=color, linewidth=2, head_width=0.25, head_length=0.2)
ax[0].text(-129.7, 47, 'S. Cascades', color=color, rotation=0, horizontalalignment='center', fontsize=fontsize-3, fontweight=fontweight)
ax[0].arrow(-126, 47.1, 1.3, 0, color=color, linewidth=2, head_width=0.25, head_length=0.2)
ax[0].text(-132, 64, 'N. Rockies', color=color, rotation=0, fontsize=fontsize-3, fontweight=fontweight)
ax[0].text(-122, 55, 'C. Rockies', color=color, rotation=0, fontsize=fontsize-3, fontweight=fontweight)
ax[0].text(-117.7, 47, 'S. Rockies', color=color, rotation=0, fontsize=fontsize-3, fontweight=fontweight)

### b) Mean weather conditions
sns.scatterplot(data=clusters, x='mean_annual_temp_range', y='mean_annual_precip_cumsum', 
                hue='clustName', palette=cluster_cmap_dict, alpha=1, ax=ax[1], s=15, legend=False)
ax[1].set_xlabel('Air temperature range [$^{\circ}$C]', fontsize=fontsize)
ax[1].set_ylabel('Precipitation sum [m]', fontsize=fontsize)

### c) Boxplots
sns.boxplot(fsnow_obs, x='clustName', y='fsnow_obs', hue='clustName', 
            palette=cluster_cmap_dict, showfliers=False, ax=ax[2])
ax[2].legend().remove()
ax[2].set_xticks([])
ax[2].set_xlabel('Climate cluster')
ax[2].set_ylabel('f$_{snow}$ []')

# Add text labels
text_labels = ['a', 'b', 'c']
for i, axis in enumerate(ax):
    if i==1:
        scale = 0.85
    else:
        scale = 0.9
    axis.text((axis.get_xlim()[1] - axis.get_xlim()[0])*scale + axis.get_xlim()[0],
              (axis.get_ylim()[1] - axis.get_ylim()[0])*scale + axis.get_ylim()[0],
              text_labels[i], fontsize=fontsize+4, fontweight='bold')

fig.subplots_adjust(wspace=0.4, hspace=0.15)
plt.show()

# Save figure
# fig_fn = os.path.join(figures_out_path, 'fig02_ELA_PDD_sensitivities.png')
# fig.savefig(fig_fn, dpi=300, bbox_inches='tight')
# print('Figure saved to file:', fig_fn)


## Figure S4. PDD coefficient correlations with elevation range

In [ ]:
fig = plt.figure(figsize=(8,14))
gs = matplotlib.gridspec.GridSpec(6, 2, height_ratios=[2,1,1,1,1,1])
ax = [fig.add_subplot(gs[0,:]),
      fig.add_subplot(gs[1,0]), fig.add_subplot(gs[1,1]),
      fig.add_subplot(gs[2,0]), fig.add_subplot(gs[2,1]),
      fig.add_subplot(gs[3,0]), fig.add_subplot(gs[3,1]),
      fig.add_subplot(gs[4,0]), fig.add_subplot(gs[4,1]),
      fig.add_subplot(gs[5,0]), fig.add_subplot(gs[5,1])]

# function for linear regression
def linear_regression(X,y):
    lr = LinearRegression().fit(X, y)
    score = lr.score(X, y)
    Xpred = np.linspace(np.min(X), np.max(X), 50).reshape(-1, 1)
    ypred = lr.predict(Xpred)
    return Xpred, ypred, score

# All subregions
sns.scatterplot(data=fits_obs_df, x='coef_PDD_median', y='Zrange_km', hue='clustName',
                palette=cluster_cmap_dict, ax=ax[0], legend=False, sizes=100)
X = fits_obs_df['coef_PDD_median'].values.reshape(-1, 1)
y = fits_obs_df['Zrange_km'].values
Xpred, ypred, score = linear_regression(X,y)
ax[0].plot(Xpred, ypred, '-k', alpha=0.5)
ax[0].set_ylabel('')
ax[0].set_xlabel('')
ax[0].text(ax[0].get_xlim()[0] + (ax[0].get_xlim()[1] - ax[0].get_xlim()[0]) * 0.95,
          ax[0].get_ylim()[0] + (ax[0].get_ylim()[1] - ax[0].get_ylim()[0]) * 0.1,
         f'R$^2$ = {np.round(score,3)}', ha = 'right')
ax[0].set_title('a) All regions')

# Individual subregions
text_labels = ['b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k']
for i, subregion in enumerate(subregion_order):
    fits_obs_subregion_df = fits_obs_df.loc[fits_obs_df['Subregion']==subregion]
    # coefficient vs. Zrange
    sns.scatterplot(data=fits_obs_subregion_df, x='coef_PDD_median', y='Zrange_km', hue='clustName',
                    palette=cluster_cmap_dict, ax=ax[i+1], legend=False, sizes=100)
    # linear regression
    X = fits_obs_subregion_df['coef_PDD_median'].values.reshape(-1, 1)
    y = fits_obs_subregion_df['Zrange_km'].values
    Xpred, ypred, score = linear_regression(X,y)
    ax[i+1].plot(Xpred, ypred, '-k', alpha=0.5)
    ax[i+1].set_ylabel('')
    ax[i+1].set_xlabel('')
    ax[i+1].text(ax[i+1].get_xlim()[0] + (ax[i+1].get_xlim()[1] - ax[i+1].get_xlim()[0]) * 0.95,
             ax[i+1].get_ylim()[0] + (ax[i+1].get_ylim()[1] - ax[i+1].get_ylim()[0]) * 0.1,
             f'R$^2$ = {np.round(score,3)}', ha = 'right')
    ax[i+1].set_title(f'{text_labels[i]}) {subregion}')

# Add axis labels
for i in [0, 1, 3, 5, 7, 9]:
    ax[i].set_ylabel('Elevation range [km]')
for i in [9, 10]:
    ax[i].set_xlabel('$\Sigma$PDD coefficient [m/$^{\circ}$C]')
        
fig.tight_layout()
plt.show()

# Save to file
fig_fn = os.path.join(figures_out_path, 'figS4_coefPDDmedian_vs_Zrange.png')
fig.savefig(fig_fn, dpi=300, bbox_inches='tight')
print('Figure saved to file:', fig_fn)

## Figure 3. ELAs: observed/modeled comparison

In [ ]:
# Load merged monthly transient and annual ELAs 
slas_merged_monthly_fn = os.path.join(scm_path, 'analysis', 'monthly_SLAs_modeled_observed_merged.csv')
slas_merged_monthly = pd.read_csv(slas_merged_monthly_fn)
slas_merged_monthly['Date'] = pd.to_datetime(slas_merged_monthly['Date'])
slas_merged_monthly['Month'] = pd.DatetimeIndex(slas_merged_monthly['Date']).month
elas_merged_annual_fn = os.path.join(scm_path, 'analysis', 'annual_ELAs_modeled_observed_merged.csv')
elas_merged_annual = pd.read_csv(elas_merged_annual_fn)
print('Merged ELAs loaded')

# AOIs
aois_fn = os.path.join(scm_path, 'analysis', 'all_aois.shp')
aois = gpd.read_file(aois_fn)
aois[['O1Region', 'O2Region']] = aois[['O1Region', 'O2Region']].astype(int)
print('Compiled AOIs loaded')

# Load climate clusters
clusters_fn = os.path.join(scm_path, 'analysis', 'climate_clusters.csv')
clusters = pd.read_csv(clusters_fn)
print('Clusters loaded')

# Add subregion and cluster names to ELA dfs
slas_merged_monthly[['Subregion', 'clustName']] = '', ''
elas_merged_annual[['Subregion', 'clustName']] = '', ''
for rgi_id in aois['RGIId'].drop_duplicates().values:
    cluster = clusters.loc[clusters['RGIId']==rgi_id, 'clustName'].values[0]
    subregion = aois.loc[aois['RGIId']==rgi_id, 'Subregion'].values[0]
    slas_merged_monthly.loc[slas_merged_monthly['RGIId']==rgi_id, ['Subregion', 'clustName']] = subregion, cluster
    elas_merged_annual.loc[elas_merged_annual['RGIId']==rgi_id, ['Subregion', 'clustName']] = subregion, cluster 
    

In [ ]:
# Plot
fontsize=12
plt.rcParams.update({'font.size':fontsize, 'font.sans-serif': 'Arial'})
fig, ax = plt.subplots(1, 2, figsize=(10,5), gridspec_kw=dict(width_ratios=[2,1]))

# Monthly snowline differences
slas_merged_monthly['SLA_mod-obs_m'] = slas_merged_monthly['SLA_mod_m'] - slas_merged_monthly['SLA_obs_m']
sns.boxplot(data=slas_merged_monthly, x='Month', y='SLA_mod-obs_m', hue='clustName', hue_order=cluster_order,
            palette=cluster_cmap_dict, saturation=1, showfliers=False,
            boxprops=dict(linewidth=1, edgecolor='k'), whiskerprops=dict(linewidth=1, color='k'), 
            ax=ax[0])
ax[0].set_xticks(np.arange(0,5))
ax[0].set_xticklabels(['May', 'Jun', 'Jul', 'Aug', 'Sep'])
ax[0].set_ylim(-1500, 1500)
ax[0].set_title('a) Modeled $-$ Observed snowline altitudes')
ax[0].set_ylabel('Snowline altitude difference [m]')

# ELA differences
elas_merged_annual['ELA_mod-obs_m'] = elas_merged_annual['ELA_mod_m'] - elas_merged_annual['ELA_obs_m']
sns.kdeplot(data=elas_merged_annual, y='ELA_mod-obs_m', hue='clustName', hue_order=cluster_order, 
            palette=cluster_cmap_dict, ax=ax[1])
# Add median lines
# xmin, xmax = ax[1].get_xlim()
# for clustName in elas_merged_annual['clustName'].drop_duplicates().values:
#     med = elas_merged_annual.loc[elas_merged_annual['clustName']==clustName, 'ELA_mod-obs_m'].median()
#     ax[1].axhline(y=med, xmin=0, xmax=1, color=cluster_cmap_dict[clustName], linestyle='-', linewidth=1)
ax[1].set_ylim(-1500, 1500)
ax[1].set_ylabel('')
ax[1].set_title('b) Modeled $-$ Observed ELAs')
ax[1].set_ylabel('ELA difference [m]')
ax[1].set_xticks([])

# Add legend
handles, labels = ax[0].get_legend_handles_labels()
ax[0].legend().remove()
ax[1].legend().remove()
fig.legend(handles, labels, ncols=len(cluster_order), loc='lower center',
           bbox_to_anchor=[0.43, -0.1, 0.2, 0.2], columnspacing=0.8, handletextpad=0.25)

# Add text label and line at 0
text_labels = ['a', 'b']
for i, axis in enumerate(ax):
    axis.axhline(0, color='k')
    # axis.text((axis.get_xlim()[1]-axis.get_xlim()[0])*0.9 + axis.get_xlim()[0],
    #           (axis.get_ylim()[1]-axis.get_ylim()[0])*0.9 + axis.get_ylim()[0],
    #           text_labels[i], fontsize=fontsize+4, fontweight='bold')
    
# fig.subplots_adjust(wspace=0.2)
fig.tight_layout()
plt.show()

# Save to file
fig_fn = os.path.join(figures_out_path, 'fig03_ELAs_observed_modeled_differences.png')
fig.savefig(fig_fn, dpi=300, bbox_inches='tight')
print('Figure saved to file:', fig_fn)


In [ ]:
## Plot boxplots by subregion and cluster
fig = plt.figure(figsize=(10,10))
subfigs = fig.subfigures(nrows=2, ncols=1)
subtitles = ['Monthly transient ELAs', 'Annual ELAs']
for subfig, subtitle, df in zip(subfigs, subtitles, [slas_merged_monthly, elas_merged_annual]):
    subfig.suptitle(subtitle)
    axs = subfig.subplots(nrows=1, ncols=2)
    for axis, group in zip(axs, [group1, group2]):
        df_group = df.loc[df['Subregion'].isin(group)]
        sns.boxplot(df_group, x='ELA_obs-mod_m', y='Subregion', showfliers=False,
                    hue='clustName', palette=cluster_cmap_dict, order=group, hue_order=cluster_order,
                    boxprops=dict(linewidth=1), ax=axis)
        axis.legend().remove()
        axis.yaxis.set_minor_locator(matplotlib.ticker.FixedLocator(np.arange(-0.5, 9.5, step=1)))
        axis.yaxis.grid(True, which='minor')
        axis.set_ylabel('')
        axis.set_xlabel('Remotely-sensed $-$ Modeled [m]')
        axis.set_xlim(-2000, 2000)
        axis.axvline(0, color='grey')

fig.subplots_adjust(wspace=0.4)
plt.show()

In [ ]:
# SLA stats
print(slas_merged_monthly.groupby(['Month'])['ELA_obs-mod_m'].describe())
slas_merged_monthly.groupby(['clustName', 'Month'])['ELA_obs-mod_m'].describe()



In [ ]:
# ELA stats
print(elas_merged_annual['ELA_obs-mod_m'].describe())
elas_merged_annual.groupby('clustName')['ELA_obs-mod_m'].describe().sort_values(by='50%')

In [ ]:
## Plot monthly histograms
slas_merged_monthly['Month'] = pd.DatetimeIndex(slas_merged_monthly['Date']).month
slas_merged_monthly['Difference'] = slas_merged_monthly['ELA_obs_m'] - slas_merged_monthly['ELA_mod_m']

fig, ax = plt.subplots()
sns.boxplot(data=slas_merged_monthly, x='Month', y='Difference', hue='clustName', 
            palette=cluster_cmap_dict, showfliers=False, ax=ax, saturation=1)
sns.move_legend(ax, loc='center right', bbox_to_anchor=[1.35, 0.5, 0.2, 0.2], title='Climate class')
ax.set_ylabel('Snowline elevation difference [m]\n(Remotely-sensed$-$Modeled)')
ax.set_xticks(np.arange(0,6))
ax.set_xticklabels(['May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct'])
ax.axhline(0, color='k')
plt.show()

## Plot weekly histograms
slas_merged_monthly['Week'] = slas_merged_monthly['Date'].dt.isocalendar().week
fig, ax = plt.subplots()
sns.boxplot(data=slas_merged_monthly, x='Week', y='Difference', hue='clustName', 
            palette=cluster_cmap_dict, showfliers=False, ax=ax, saturation=1)
sns.move_legend(ax, loc='center right', bbox_to_anchor=[1.35, 0.5, 0.2, 0.2], title='Climate class')
ax.set_ylabel('Snowline elevation difference [m]\n(Remotely-sensed$-$Modeled)')
# ax.set_xticks(np.arange(0,6))
# ax.set_xticklabels(['May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct'])
ax.axhline(0, color='k')
plt.show()



## Figure S1. Sites distributions

In [ ]:
# Load analyzed glacier boundaires
aois_fn = os.path.join(scm_path, 'analysis', 'all_aois.shp')
aois = gpd.read_file(aois_fn)
cols = ['O1Region', 'O2Region', 'Zmed', 'Aspect', 'Slope', 'Area']
for col in cols:
    aois[col] = aois[col].astype(float)
aois['Subregion'] = ''
for o1region, o2region in aois[['O1Region', 'O2Region']].drop_duplicates().values:
    subregion_name, color = f.determine_subregion_name_color(o1region, o2region)
    aois.loc[(aois['O1Region']==o1region) & (aois['O2Region']==o2region), 'Subregion'] = subregion_name

    
# Load all glacier boundaries in O1 regions 1 and 2
rgi_path = '/Volumes/LaCie/raineyaberle/Research/PhD/GIS_data/RGI/'
rgi_fns = ['01_rgi60_Alaska/01_rgi60_Alaska.shp',
           '02_rgi60_WesternCanadaUS/02_rgi60_WesternCanadaUS.shp']
rgi = gpd.GeoDataFrame()
for rgi_fn in rgi_fns:
    file = gpd.read_file(os.path.join(rgi_path, rgi_fn))
    rgi = pd.concat([rgi, file])
rgi[['O1Region', 'O2Region']] = rgi[['O1Region', 'O2Region']].astype(int)
# Add column for subregion name
rgi['Subregion'] = ''
for o1region, o2region in rgi[['O1Region', 'O2Region']].drop_duplicates().values:
    subregion_name, color = f.determine_subregion_name_color(o1region, o2region)
    rgi.loc[(rgi['O1Region']==o1region) & (rgi['O2Region']==o2region), 'Subregion'] = subregion_name


In [ ]:
# Define plotting variables
columns = ['Zmed', 'Aspect', 'Slope', 'Area']
xlabels = ['Median elevation [m]', 'Aspect [degrees]', 'Slope [degrees]', 'Area [km$^2$]']
bins_list = [np.linspace(0, 361, num=20), # Aspect
             np.linspace(0, 51, num=20), # Slope
             np.linspace(0, 300, num=50)] # Area

# Set up figure
plt.rcParams.update({'font.sans-serif': 'Arial', 'font.size': 12})
fig, ax = plt.subplots(len(subregion_order)+1, 4, figsize=(12, (len(subregion_order)+1)*1.5))
aois_color = '#b35806'

# All subregions
for j, (column, xlabel) in enumerate(zip(columns, xlabels)):
    if column=='Zmed': 
        bins = np.linspace(rgi['Zmed'].min(), rgi['Zmed'].max(), num=20)
    else:
        bins  = bins_list[j-1]
    ax[0,j].hist(rgi[column].values, bins=bins, facecolor='k', alpha=0.6)
    ax[0,j].set_title(xlabel)
    ax2 = ax[0,j].twinx()
    ax2.hist(aois[column].values, bins=bins, facecolor=aois_color, alpha=0.6)
    ax2.set_yticks(ax2.get_yticks())
    ax2.set_yticklabels(ax2.get_yticklabels(), color=aois_color)
    ax2.spines['right'].set_color(aois_color)
    ax2.tick_params(axis='y', color=aois_color)
ax[0,0].set_ylabel('All regions', fontweight='bold')

# Individual subregions
for i, subregion in enumerate(subregion_order):
    # Subset glaciers
    aois_subregion = aois.loc[aois['Subregion']==subregion]
    rgi_subregion = rgi.loc[rgi['Subregion']==subregion]

    # Plot all glaciers in subregion
    for j, (column, xlabel) in enumerate(zip(columns, xlabels)):
        if column=='Zmed':
            bins = np.linspace(rgi_subregion['Zmed'].min(), rgi_subregion['Zmed'].max(), num=20)
        else:
            bins = bins_list[j-1]
        ax[i+1,j].hist(rgi_subregion[column].values, bins=bins, facecolor='k', alpha=0.6)
        if j==0:
            ax[i+1,j].set_ylabel(subregion)
        ax2 = ax[i+1,j].twinx()
        ax2.hist(aois_subregion[column].values, bins=bins, facecolor=aois_color, alpha=0.6)
        ax2.set_yticks(ax2.get_yticks())
        ax2.set_yticklabels(ax2.get_yticklabels(), color=aois_color)
        ax2.spines['right'].set_color(aois_color)
        ax2.tick_params(axis='y', color=aois_color)

fig.tight_layout()
plt.show()

# Save figure
fig_fn = os.path.join(figures_out_path, 'figS1_site_distributions.png')
fig.savefig(fig_fn, dpi=300, bbox_inches='tight')
print('Figure saved to file:', fig_fn)

## Figure S3. Regression scores

In [ ]:
# -----Load linear fits
fits_obs_fn = os.path.join(scm_path, 'analysis', 'linear_fit_observed_monthly_ela_pdd_snowfall.csv')
fits_obs_df = pd.read_csv(fits_obs_fn)
# Add center longitude and latitude coordinates from AOIs
aois_fn = os.path.join(scm_path, 'analysis', 'all_aois.shp')
aois = gpd.read_file(aois_fn)
fits_obs_df = pd.merge(fits_obs_df, aois[['RGIId', 'CenLon', 'CenLat']], on='RGIId')
# Add weather clusters
clusters = pd.read_csv(clusters_fn)
fits_obs_df = pd.merge(fits_obs_df, clusters[['RGIId', 'clustName']], on='RGIId')
print('Linear fits loaded')

# -----Load RGI O2 Regions
rgi_O2_fn = os.path.join(scm_path, '..', 'GIS_data', 'RGI', 'RGIv7_02Regions', 
                                'RGI2000-v7.0-o2regions-Alaska-westernCanadaUS_clipped_to_country_outlines.shp')
rgi_O2 = gpd.read_file(rgi_O2_fn)
# remove Brooks Range
rgi_O2 = rgi_O2.loc[rgi_O2['o2region']!='01-01']
# add subregion name and color column
rgi_O2[['Subregion', 'color']] = '', ''
for o1o2 in rgi_O2['o2region'].drop_duplicates().values:
    o1 = int(o1o2[0:2])
    o2 = int(o1o2[3:])
    subregion_name, color = f.determine_subregion_name_color(o1, o2)
    rgi_O2.loc[rgi_O2['o2region']==o1o2, 'Subregion'] = subregion_name
    rgi_O2.loc[rgi_O2['o2region']==o1o2, 'color'] = color
print('RGI O2 regions loaded from file')

# -----Load GTOPO30
gtopo_fn = '/Users/raineyaberle/Research/PhD/GIS_data/GTOPO30_clip.tif'
gtopo = rxr.open_rasterio(gtopo_fn)
gtopo = xr.where(gtopo==-32768, np.nan, gtopo)
print('GTOPO30 loaded from file')

In [ ]:
fontsize=16
plt.rcParams.update({'font.size': fontsize, 'font.sans-serif': 'Arial'})
fig, ax = plt.subplots(1, 2, figsize=(14,6), gridspec_kw={'width_ratios': [1,1.4]})

# Histogram of regression scores
sns.histplot(data=fits_obs_df, x='score_mean', hue='clustName', hue_order=cluster_order, 
             palette=cluster_cmap_dict, multiple='stack', bins=20, legend=False, ax=ax[0])
ax[0].set_xlabel('Mean regression score')

# Hillshade
ls = matplotlib.colors.LightSource(azdeg=90, altdeg=45)
ax[1].imshow(ls.hillshade(gtopo.data[0], vert_exag=0.002), cmap='gray', alpha=0.5,
             extent=(np.min(gtopo.x.data), np.max(gtopo.x.data), 
                     np.min(gtopo.y.data), np.max(gtopo.y.data)))
# RGI O2 region outlines
color = '#525252'
rgi_O2.plot(ax=ax[1], alpha=1.0, facecolor='None', edgecolor=color, linewidth=1)
# Map of regression scores
sns.scatterplot(data=fits_obs_df, x='CenLon', y='CenLat', hue='clustName', hue_order=cluster_order, 
                palette=cluster_cmap_dict, size='score_mean', sizes=(1,75), edgecolor='#525252', 
                linewidth=1, legend='brief', ax=ax[1])
handles, labels = ax[1].get_legend_handles_labels()
# weather class legend
fig.legend(handles[1:6], labels[1:6], loc='upper center', ncols=5, markerscale=2, 
           bbox_to_anchor=[0.4, 0.88, 0.2, 0.2], frameon=False, handletextpad=0.4)
# marker sizes legend
handles, labels = handles[7:], labels[7:]
leg = ax[1].legend(handles, labels, title="Mean regression score", 
                   loc='lower left', alignment='left', bbox_to_anchor=[0.2, 0.01, 0.2, 0.2])
ax[1].set_yticks(np.linspace(45, 65, num=6))
ax[1].set_xlim(-167, -112)
ax[1].set_ylim(46, 67)
ax[1].set_xlabel('Longitude')
ax[1].set_ylabel('Latitude')
ax[1].set_aspect(2)

# Add text labels
ax[0].text(ax[0].get_xlim()[0] + (ax[0].get_xlim()[1] - ax[0].get_xlim()[0])*0.9,
           ax[0].get_ylim()[0] + (ax[0].get_ylim()[1] - ax[0].get_ylim()[0])*0.9,
           'a', fontweight='bold', fontsize=fontsize+4)
ax[1].text(ax[1].get_xlim()[0] + (ax[1].get_xlim()[1] - ax[1].get_xlim()[0])*0.93,
           ax[1].get_ylim()[0] + (ax[1].get_ylim()[1] - ax[1].get_ylim()[0])*0.9,
           'b', fontweight='bold', fontsize=fontsize+4)

fig.tight_layout()
plt.show()

# Save to file
fig_fn = os.path.join(figures_out_path, 'figS3_regression_scores.png')
fig.savefig(fig_fn, dpi=300, bbox_inches='tight')
print('Figure saved to file:', fig_fn)

fits_obs_df[['score_mean']].describe()

In [ ]:
fits_obs_df.loc[fits_obs_df['score_mean'] < 0.1]

## Table S1. Study sites with subregion and weather cluster

In [ ]:
aois = gpd.read_file(aois_fn)
aois.rename(columns={'Subregion': 'Subregion name'}, inplace=True)
aois[['O1Region', 'O2Region']] = aois[['O1Region', 'O2Region']].astype(int)

# Add weather cluster
clusters = pd.read_csv(clusters_fn)
aois['Weather cluster'] = ''
for rgi_id in aois['RGIId'].drop_duplicates().values:
    aois.loc[aois['RGIId']==rgi_id, 'Weather cluster'] = clusters.loc[clusters['RGIId']==rgi_id, 'clustName']
aois.sort_values(by=['O1Region', 'O2Region'], inplace=True)

# Format as LaTeX table
columns = ['RGIId', 'O1Region', 'O2Region', 'Subregion name', 'Weather cluster']
aois = aois[columns]

# Save as Excel sheet
aois_xl_fn = aois_fn.replace('all_aois.shp', 'TableS1_study_sites.xlsx')
aois.to_excel(aois_xl_fn, index=False)
print('Table saved as Excel spreadsheet:', aois_xl_fn)
aois


## AGU24 abstract figure

In [ ]:
cmap = sns.color_palette('mako', n_colors=len(subregion_order)+2)

# -----Load median AARs for all sites
min_snow_cover_stats_fn = os.path.join(scm_path, 'results', 'min_snow_cover_stats.csv')
min_snow_cover_stats = pd.read_csv(min_snow_cover_stats_fn)
# Sort subregions
min_snow_cover_stats['order'] = ''
for i, subregion in enumerate(subregion_order):
    min_snow_cover_stats.loc[min_snow_cover_stats['Subregion']==subregion, 'order'] = i
    min_snow_cover_stats.loc[min_snow_cover_stats['Subregion']==subregion, 'color'] = matplotlib.colors.to_hex(cmap[i])
min_snow_cover_stats = min_snow_cover_stats.sort_values(by='order')
print('Median AARs loaded from file')

# # -----Load RGI O2 Regions
# rgi_O2_fn = os.path.join(scm_path, '..', 'GIS_data', 'RGI', 'RGIv7_02Regions', 
#                                 'RGI2000-v7.0-o2regions-Alaska-westernCanadaUS_clipped_to_country_outlines.shp')
# rgi_O2 = gpd.read_file(rgi_O2_fn)
# # remove Brooks Range
# rgi_O2 = rgi_O2.loc[rgi_O2['o2region']!='01-01']
# # add subregion name and color column
# rgi_O2[['Subregion', 'color']] = '', ''
# for i, o1o2 in enumerate(rgi_O2['o2region'].drop_duplicates().values):
#     o1 = int(o1o2[0:2])
#     o2 = int(o1o2[3:])
#     subregion_name, color = f.determine_subregion_name_color(o1, o2)
#     rgi_O2.loc[rgi_O2['o2region']==o1o2, 'Subregion'] = subregion_name
#     rgi_O2.loc[rgi_O2['o2region']==o1o2, 'color'] = dict(min_snow_cover_stats[['Subregion', 'color']].drop_duplicates().values)[subregion]
# print('RGI O2 regions loaded from file')

# # -----Load GTOPO30
# gtopo_fn = '/Users/raineyaberle/Research/PhD/GIS_data/GTOPO30_clip.tif'
# gtopo = rxr.open_rasterio(gtopo_fn)
# gtopo = xr.where(gtopo==-32768, np.nan, gtopo)
# print('GTOPO30 loaded from file')

# # -----Load classified image
# site_name = 'RGI60-01.00037'
# im_classified_fn = f'/Volumes/LaCie/raineyaberle/Research/PhD/snow_cover_mapping/study-sites/{site_name}/imagery/classified/20230802T152742_RGI60-01.00037_Sentinel-2_SR_classified.nc'
# im_classified = xr.open_dataset(im_classified_fn)
# print('Classified image loaded')

# # -----Load classified images colormap
# import json
# datasets_dict_fn = '/Users/raineyaberle/Research/PhD/snow_cover_mapping/snow-cover-mapping/inputs-outputs/datasets_characteristics.json'
# datasets_dict = json.load(open(datasets_dict_fn))
# cmap_classified = matplotlib.colors.ListedColormap(datasets_dict['classified_image']['class_colors'].values())

# # -----Load Sentinel-2 image from GEE
# import math
# import wxee as wx
# import geedim as gd
# import ee
# ee.Initialize()
# def convert_wgs_to_utm(lon: float, lat: float):
#     utm_band = str((math.floor((lon + 180) / 6) % 60) + 1)
#     if len(utm_band) == 1:
#         utm_band = '0' + utm_band
#     if lat >= 0:
#         epsg_code = '326' + utm_band
#         return epsg_code
#     epsg_code = '327' + utm_band
#     return epsg_code
# # Load AOI
# aoi_fn = glob.glob(f'/Volumes/LaCie/raineyaberle/Research/PhD/snow_cover_mapping/study-sites/{site_name}/AOIs/*.shp')[0]
# aoi = gpd.read_file(aoi_fn)
# aoi_bounds = aoi.geometry[0].bounds
# region = ee.Geometry.Polygon([[aoi_bounds[0], aoi_bounds[1]], 
#                               [aoi_bounds[2], aoi_bounds[1]],
#                               [aoi_bounds[2], aoi_bounds[3]],
#                               [aoi_bounds[0], aoi_bounds[3]],
#                               [aoi_bounds[0], aoi_bounds[1]]])
# # Load image collection
# im_col = gd.MaskedCollection.from_name('COPERNICUS/S2_SR_HARMONIZED').search(start_date='2023-08-01',
#                                                                              end_date='2023-08-03',
#                                                                              region=region,
#                                                                              mask=True)
# im_ee = im_col.ee_collection.first()
# im_ee = im_ee.clip(region)
# im_ee = im_ee.select(['B4', 'B3', 'B2'])
# # Convert to xarray.Dataset
# im_xr = im_ee.wx.to_xarray(scale=30, crs='EPSG:4326')
# im_xr = xr.where(im_xr==im_xr.attrs['_FillValue'], np.nan, im_xr / 1e4)
# im_xr = im_xr.rio.write_crs('EPSG:4326')
# print('Sentinel-2 SR image loaded')

# # Reproject AOI and images to optimal UTM zone
# epsg_utm = convert_wgs_to_utm(aoi.geometry[0].centroid.coords.xy[0][0], aoi.geometry[0].centroid.coords.xy[1][0])
# aoi_utm = aoi.to_crs(f'EPSG:{epsg_utm}')
# im_xr = im_xr.rio.reproject(f'EPSG:{epsg_utm}')
# im_classified = im_classified.rio.write_crs("EPSG:4326")
# im_classified = im_classified.rio.reproject(f'EPSG:{epsg_utm}')
# im_classified = xr.where(im_classified < 1, np.nan, im_classified)

In [ ]:
# Set up figure
fontsize=14
plt.rcParams.update({
    "font.size": fontsize,
    "font.sans-serif": "Arial",
    # "font.family": "sans-serif",
    # "font.sans-serif": "Computer Modern Sans Serif",
    "text.usetex": False
})

fig = plt.figure(figsize=(12,12))
gs = matplotlib.gridspec.GridSpec(2, 2, figure=fig, height_ratios=[1,2])
ax = [fig.add_subplot(gs[2:]),
      fig.add_subplot(gs[0]),
      fig.add_subplot(gs[1])]

# ----- Study sites
# GTOPO hillshade
ls = matplotlib.colors.LightSource(azdeg=90, altdeg=45)
ax[0].imshow(ls.hillshade(gtopo.data[0], vert_exag=0.002), cmap='gray', alpha=0.5,
             extent=(np.min(gtopo.x.data), np.max(gtopo.x.data), 
                     np.min(gtopo.y.data), np.max(gtopo.y.data)))
# RGI O2 region outlines
color = '#525252'
rgi_O2.plot(ax=ax[0], alpha=1.0, facecolor='None', edgecolor=color, linewidth=1)
ax[0].set_yticks(np.linspace(45, 65, num=6))
ax[0].set_xlim(-167, -112)
ax[0].set_ylim(46, 67)
ax[0].set_xlabel('Longitude')
ax[0].set_ylabel('Latitude')
ax[0].set_aspect(2.2)
# Median AARs
sns.scatterplot(data=min_snow_cover_stats, x='CenLon', y='CenLat', edgecolor='w', linewidth=0.5, 
                hue='Subregion', hue_order=subregion_order, palette=dict(min_snow_cover_stats[['Subregion', 'color']].drop_duplicates().values), 
                alpha=1, size='AAR_P50_min', sizes=(2,100), ax=ax[0])
handles, labels = ax[0].get_legend_handles_labels()
Ikeep = np.argwhere(['0.' in x for x in np.array(labels)]).flatten()
handles, labels = [handles[i] for i in Ikeep], [labels[i] for i in Ikeep] 
ax[0].legend(handles, labels, loc='lower left', title='2013–2023 median AAR', bbox_to_anchor=[0.2, 0.05, 0.2, 0.2])
# Add region labels and arrows
fontweight = 'bold'
ax[0].text(-163, 56, 'Aleutians', color=color, rotation=35, fontsize=fontsize-2, fontweight=fontweight)
ax[0].text(-158, 62.3, 'Alaska Range', color=color, rotation=0, fontsize=fontsize-2, fontweight=fontweight)
ax[0].text(-147.9, 57.8, 'W. Chugach \nMtns.', color=color, rotation=0, horizontalalignment='center', fontsize=fontsize-2, fontweight=fontweight)
ax[0].arrow(-147.6, 58.8, 0, 0.8, color=color, linewidth=2, head_width=0.25, head_length=0.2)
ax[0].text(-141.7, 57.7, 'St. Elias \nMtns.', color=color, rotation=0, horizontalalignment='center', fontsize=fontsize-2, fontweight=fontweight)
ax[0].arrow(-141.5, 58.7, 0, 0.8, color=color, linewidth=2, head_width=0.25, head_length=0.2)
ax[0].text(-139.6, 56.4, 'N. Coast \nRanges', color=color, rotation=0, horizontalalignment='center', fontsize=fontsize-2, fontweight=fontweight)
ax[0].arrow(-137.3, 56.8, 1.3, 0, color=color, linewidth=2, head_width=0.25, head_length=0.2)
ax[0].text(-133, 51.3, 'N. Cascades', color=color, rotation=0, horizontalalignment='center', fontsize=fontsize-2, fontweight=fontweight)
ax[0].arrow(-129.4, 51.4, 1.3, 0, color=color, linewidth=2, head_width=0.25, head_length=0.2)
ax[0].text(-129.7, 47, 'S. Cascades', color=color, rotation=0, horizontalalignment='center', fontsize=fontsize-2, fontweight=fontweight)
ax[0].arrow(-126, 47.1, 1.3, 0, color=color, linewidth=2, head_width=0.25, head_length=0.2)
ax[0].text(-132, 64, 'N. Rockies', color=color, rotation=0, fontsize=fontsize-2, fontweight=fontweight)
ax[0].text(-122, 55, 'C. Rockies', color=color, rotation=0, fontsize=fontsize-2, fontweight=fontweight)
ax[0].text(-117.7, 47, 'S. Rockies', color=color, rotation=0, fontsize=fontsize-2, fontweight=fontweight)
# Example site location
min_snow_cover_stats_site = min_snow_cover_stats.loc[min_snow_cover_stats['RGIId']==site_name]
ax[0].plot(min_snow_cover_stats_site['CenLon'], min_snow_cover_stats_site['CenLat'], '*', 
           markeredgecolor='k', markerfacecolor='#e7298a', markersize=15, linewidth=2)
    
# -----b) Sentinel-2 image
ax[1].imshow(np.dstack([im_xr.B4.data[0], im_xr.B3.data[0], im_xr.B2.data[0]]),
             extent=(np.min(im_xr.x.data)/1e3, np.max(im_xr.x.data)/1e3, 
                     np.min(im_xr.y.data)/1e3, np.max(im_xr.y.data)/1e3))
ax[1].set_xlabel('Easting [km]')
ax[1].set_ylabel('Northing [km]')

# -----c) Classified image
ax[2].imshow(im_classified.classified.data[0], cmap=cmap_classified, clim=(1,5),
             extent=(np.min(im_classified.x.data)/1e3, np.max(im_classified.x.data)/1e3,
                     np.min(im_classified.y.data)/1e3, np.max(im_classified.y.data)/1e3))
ax[2].plot(np.divide(aoi_utm.geometry[0].exterior.coords.xy[0], 1e3), np.divide(aoi_utm.geometry[0].exterior.coords.xy[1], 1e3),
           '-k', label='Glacier boundary')
ax[2].set_xlabel('Easting [km]')
# dummy points for legend
ax[2].plot(0, 0, 's', color=cmap_classified(0), label='Snow')
ax[2].plot(0, 0, 's', color=cmap_classified(1), label='Shadowed snow')
ax[2].plot(0, 0, 's', color=cmap_classified(2), label='Ice/firn')
ax[2].plot(0, 0, 's', color=cmap_classified(3), label='Rock/debris')
ax[2].plot(0, 0, 's', color=cmap_classified(4), label='Water')
ax[2].set_xlim(ax[1].get_xlim())
ax[2].set_ylim(ax[1].get_ylim())
handles, labels = ax[2].get_legend_handles_labels()
fig.legend(handles, labels, loc='upper center', ncols=6, markerscale=2, frameon=False)

# Plot AOI
for axis in ax[1:]:
    axis.plot(np.divide(aoi_utm.geometry[0].exterior.coords.xy[0], 1e3), 
              np.divide(aoi_utm.geometry[0].exterior.coords.xy[1], 1e3),
              '-k', linewidth=1, label='Glacier boundary')
    axis.set_yticks(np.arange(7030, 7046, step=5))

# Add text labels
text_labels = ['c', 'a', 'b']
for i in range(0, len(ax)):
    ax[i].text((ax[i].get_xlim()[1] - ax[i].get_xlim()[0]) * 0.9 + ax[i].get_xlim()[0],
               (ax[i].get_ylim()[1] - ax[i].get_ylim()[0]) * 0.85 + ax[i].get_ylim()[0],
                text_labels[i], fontweight='bold', fontsize=fontsize+4, horizontalalignment='center',
              bbox=dict(facecolor='w', edgecolor='None', pad=3))

# # Add caption
# caption = (r"\noindent\textbf{Figure 1. a)} Sentinel-2 surface reflectance image captured 2023-08-02 for one glacier (Randolph Glacier Inventory ID = 1.00037) \\"
#             r"and the associated \textbf{b)} classified image generated from the automated snow detection pipeline. \textbf{c)} Map of the study glacier locations, \\"
#             r"with marker sizes indicating the median accumulation area ratio (AAR) for the 2013–2023 study period. The maroon start marks the \\"
#             r"location of the example glacier shown in panels \textbf{a} and \textbf{b}." )
# fig.text(0.05, -0.02, caption, ha='left', wrap=True, fontsize=fontsize+1)

fig.tight_layout()
plt.show()

# Save figure
fig_fn = os.path.join(figures_out_path, 'agu24_abstract_figure.png')
fig.savefig(fig_fn, dpi=300, bbox_inches='tight')
print('Figure saved to file:', fig_fn)


## Snow cover GIF

In [ ]:
import glob

# Load inputs
rgi_id = "RGI60-01.00312"
aoi_fn = os.path.join(scm_path, 'study-sites', rgi_id, 'AOIs', f"{rgi_id}_outline.shp")
aoi = gpd.read_file(aoi_fn)
im_classified_fns = sorted(glob.glob(os.path.join(scm_path, 'study-sites', rgi_id, 'imagery', 'classified', '*.nc')))
scs_fn = os.path.join(scm_path, 'study-sites', rgi_id, f"{rgi_id}_snow_cover_stats.csv")
scs = pd.read_csv(scs_fn)
scs['datetime'] = pd.to_datetime(scs['datetime'], format='mixed')
# subset to 2019
im_classified_fns = [x for x in im_classified_fns if int(os.path.basename(x)[0:4]) == 2019] 
scs = scs.loc[scs['datetime'].dt.year == 2019]
out_path = os.path.join(figures_out_path, 'timeseries_gif')
if not os.path.exists(out_path):
    os.mkdir(out_path)
    print('Made directory for outputs:', out_path)

# Define colormap for classified images
cmap_dict = {"Snow": "#4eb3d3",  "Shadowed_snow": "#636363", "Ice": "#084081", "Rock": "#fe9929", "Water": "#252525"}
colors = []
for key in list(cmap_dict.keys()):
    color = list(matplotlib.colors.to_rgb(cmap_dict[key]))
    if key=='Rock':
        color += [0.5]
    colors.append(color)
        
cmap = matplotlib.colors.ListedColormap(colors)
cmap

In [ ]:
### Download images for 2019 ###

import math
import ee
import geedim as gd
import datetime
from rasterio.features import geometry_mask

def convert_wgs_to_utm(lon: float, lat: float):
    utm_band = str((math.floor((lon + 180) / 6) % 60) + 1)
    if len(utm_band) == 1:
        utm_band = '0' + utm_band
    if lat >= 0:
        epsg_code = 'EPSG:326' + utm_band
        return epsg_code
    epsg_code = 'EPSG:327' + utm_band
    return epsg_code
    
def query_gee_for_imagery_run_pipeline(dataset, aoi_utm, date_start, date_end,
                                       month_start, month_end, site_name, 
                                       mask_clouds=True, cloud_cover_max=70, aoi_coverage=70, im_out_path=None,
                                       verbose=False, im_download=False):

    # -----Grab optimal UTM zone from AOI CRS
    epsg_utm = str(aoi_utm.crs.to_epsg())

    # -----Reformat AOI for image filtering
    # reproject CRS from AOI to WGS
    aoi_wgs = aoi_utm.to_crs('EPSG:4326')
    # prepare AOI for querying geedim (AOI bounding box)
    region = {'type': 'Polygon',
              'coordinates': [[[aoi_wgs.geometry.bounds.minx[0], aoi_wgs.geometry.bounds.miny[0]],
                               [aoi_wgs.geometry.bounds.maxx[0], aoi_wgs.geometry.bounds.miny[0]],
                               [aoi_wgs.geometry.bounds.maxx[0], aoi_wgs.geometry.bounds.maxy[0]],
                               [aoi_wgs.geometry.bounds.minx[0], aoi_wgs.geometry.bounds.maxy[0]],
                               [aoi_wgs.geometry.bounds.minx[0], aoi_wgs.geometry.bounds.miny[0]]
                               ]]}

    # -----Define function to query GEE for imagery
    def query_gee(dataset, date_start, date_end, region, cloud_cover_max, mask_clouds):
        if dataset == 'Landsat8':
            # Landsat 8
            im_col_gd = gd.MaskedCollection.from_name('LANDSAT/LC08/C02/T1_L2').search(start_date=date_start,
                                                                                       end_date=date_end,
                                                                                       region=region,
                                                                                       cloudless_portion=100 - cloud_cover_max,
                                                                                       mask=mask_clouds)
        elif dataset == 'Landsat9':
            # Landsat 9
            im_col_gd = gd.MaskedCollection.from_name('LANDSAT/LC09/C02/T1_L2').search(start_date=date_start,
                                                                                       end_date=date_end,
                                                                                       region=region,
                                                                                       cloudless_portion=100 - cloud_cover_max,
                                                                                       mask=mask_clouds)
        elif dataset == 'Sentinel-2_TOA':
            im_col_gd = gd.MaskedCollection.from_name('COPERNICUS/S2_HARMONIZED').search(start_date=date_start,
                                                                                         end_date=date_end,
                                                                                         region=region,
                                                                                         cloudless_portion=100 - cloud_cover_max,
                                                                                         mask=mask_clouds)

        elif dataset == 'Sentinel-2_SR':
            im_col_gd = gd.MaskedCollection.from_name('COPERNICUS/S2_SR_HARMONIZED').search(start_date=date_start,
                                                                                            end_date=date_end,
                                                                                            region=region,
                                                                                            cloudless_portion=100 - cloud_cover_max,
                                                                                            mask=mask_clouds)
        else:
            print("'dataset' variable not recognized. Please set to 'Landsat', 'Sentinel-2_TOA', or 'Sentinel-2_SR'. "
                  "Exiting...")
            return 'N/A'

        return im_col_gd

    # -----Define function to filter image IDs by month range
    def filter_im_ids_month_range(im_ids, im_dts, month_start, month_end):
        i = [int(ii) for ii in np.arange(0, len(im_dts)) if
             (im_dts[ii].month >= month_start) and (im_dts[ii].month <= month_end)]  # indices of images to keep
        im_ids, im_dts = [im_ids[ii] for ii in i], [im_dts[ii] for ii in i]  # subset of image IDs and datetimes
        # return 'N/A' if no images remain after filtering by month range
        if len(im_dts) < 1:
            return 'N/A', 'N/A'
        return im_ids, im_dts

    # -----Define function to couple image IDs captured within the same hour for mosaicking
    def image_mosaic_ids(im_col_gd):
        # Grab image properties, IDs, and datetimes from image collection
        properties = im_col_gd.properties
        ims = dict(properties).keys()
        im_ids = [properties[im]['system:id'] for im in ims]
        # return if no images found
        if len(im_ids) < 1:
            return 'N/A', 'N/A'
        im_dts = np.array(
            [datetime.datetime.utcfromtimestamp(properties[im]['system:time_start'] / 1000) for im in ims])

        # Remove image datetimes and IDs outside the specified month range
        im_ids, im_dts = filter_im_ids_month_range(im_ids, im_dts, month_start, month_end)

        # Grab all unique hours in image datetimes
        hours = np.array(im_dts, dtype='datetime64[h]')
        unique_hours = sorted(set(hours))

        # Create list of IDs for each unique hour
        im_mosaic_ids_list, im_mosaic_dts_list = [], []
        for unique_hour in unique_hours:
            i = list(np.ravel(np.argwhere(hours == unique_hour)))
            im_ids_list_hour = [im_ids[ii] for ii in i]
            im_mosaic_ids_list.append(im_ids_list_hour)
            im_dts_list_hour = [im_dts[ii] for ii in i]
            im_mosaic_dts_list.append(im_dts_list_hour)

        return im_mosaic_ids_list, im_mosaic_dts_list

    # -----Define function for extracting valid image IDs
    def extract_valid_image_ids(ds, date_start, date_end, region, cloud_cover_max, mask_clouds):
        # Initialize list of date ranges for querying
        date_ranges = [(date_start, date_end)]
        # Initialize list of error dates
        error_dates = []
        # Initialize error flag
        error_occurred = True
        # Iterate until no errors occur
        while error_occurred:
            error_occurred = False  # Reset the error flag at the beginning of each iteration
            try:
                # Initialize list of image collections
                im_col_gd_list = []
                # Iterate over date ranges
                for date_range in date_ranges:
                    # Query GEE for imagery
                    im_col_gd = query_gee(ds, date_range[0], date_range[1], region, cloud_cover_max, mask_clouds)
                    properties = im_col_gd.properties  # Error will occur here if an image is inaccessible!
                    im_col_gd_list.append(im_col_gd)
                # Initialize list of filtered image IDs and datetimes
                im_mosaic_ids_list_full, im_mosaic_dts_list_full = [], []  # Initialize lists of
                # Filter image IDs for month range and couple IDs for mosaicking
                for im_col_gd in im_col_gd_list:
                    im_mosaic_ids_list, im_mosaic_dts_list = image_mosaic_ids(im_col_gd)
                    if type(im_mosaic_ids_list) is str:
                        return 'N/A', 'N/A'
                    # append to list
                    im_mosaic_ids_list_full = im_mosaic_ids_list_full + im_mosaic_ids_list
                    im_mosaic_dts_list_full = im_mosaic_dts_list_full + im_mosaic_dts_list

                return im_mosaic_ids_list_full, im_mosaic_dts_list_full

            except Exception as e:
                error_id = str(e).split('ID=')[1].split(')')[0]
                print(f"Error querying GEE for {str(error_id)}")

                # Parse the error date from the exception message (replace this with your actual parsing logic)
                error_date = datetime.datetime.strptime(error_id[0:8], '%Y%m%d')
                error_dates.append(error_date)

                # Update date ranges excluding the problematic date
                date_starts = [date_start] + [str(error_date + datetime.timedelta(days=1))[0:10] for error_date in
                                              error_dates]
                date_ends = [str(error_date - datetime.timedelta(days=1))[0:10] for error_date in error_dates] + [
                    date_end]
                date_ranges = list(zip(date_starts, date_ends))

                # Set the error flag to indicate that an error occurred
                error_occurred = True

    # -----Apply functions
    if dataset == 'Landsat':  # must run Landsat 8 and 9 separately
        im_ids_list_8, im_dts_list_8 = extract_valid_image_ids('Landsat8', date_start, date_end, region,
                                                               cloud_cover_max, mask_clouds)
        im_ids_list_9, im_dts_list_9 = extract_valid_image_ids('Landsat9', date_start, date_end, region,
                                                               cloud_cover_max, mask_clouds)
        if (type(im_ids_list_8) is str) and (type(im_ids_list_9) is str):
            im_ids_list, im_dts_list = 'N/A', 'N/A'
        elif type(im_ids_list_9) is str:
            im_ids_list, im_dts_list = im_ids_list_8, im_dts_list_8
        elif type(im_ids_list_8) is str:
            im_ids_list, im_dts_list = im_ids_list_9, im_dts_list_9
        else:
            im_ids_list = im_ids_list_8 + im_ids_list_9
            im_dts_list = im_dts_list_8 + im_dts_list_9
    else:
        im_ids_list, im_dts_list = extract_valid_image_ids(dataset, date_start, date_end, region, cloud_cover_max,
                                                           mask_clouds)

    # -----Check if any images were found after filtering
    if type(im_ids_list) is str:
        print('No images found or error in one or more image IDs, exiting...')
        return 'N/A'

    if dataset=='Landsat':
        res = 30
        image_scalar = 36363.63636363636
        no_data_value = 0
    elif 'Sentinel-2' in dataset:
        res = 10
        image_scalar = 1e4
        no_data_value = -9999

    # -----Create xarray.Datasets from list of image IDs
    # loop through image IDs
    for i in tqdm(range(0, len(im_ids_list))):

        # subset image IDs and image datetimes
        im_ids, im_dts = im_ids_list[i], im_dts_list[i]

        # make directory for outputs (out_path) if it doesn't exist
        if not os.path.exists(im_out_path):
            os.mkdir(im_out_path)
            print('Made directory for image downloads: ' + im_out_path)
            
        # define filename
        if len(im_dts) > 1:
            im_fn = dataset + '_' + str(im_dts[0]).replace('-', '')[0:8] + '_MOSAIC.tif'
        else:
            im_fn = dataset + '_' + str(im_dts[0]).replace('-', '')[0:8] + '.tif'
            
        # check file does not already exist in directory, download
        if not os.path.exists(os.path.join(im_out_path, im_fn)):
            # create list of MaskedImages from IDs
            im_gd_list = [gd.MaskedImage.from_id(im_id) for im_id in im_ids]
            # combine into new MaskedCollection
            im_collection = gd.MaskedCollection.from_list(im_gd_list)
            # create image composite
            im_composite = im_collection.composite(method=gd.CompositeMethod.q_mosaic,
                                                   mask=mask_clouds,
                                                   region=region)
            # download to file
            im_composite.download(os.path.join(im_out_path, im_fn),
                                  region=region,
                                  scale=res,
                                  crs='EPSG:4326',
                                  dtype='int16',
                                  bands=im_composite.refl_bands)

    return

# Reproject AOI to optimal UTM zone
epsg_utm = convert_wgs_to_utm(aoi.geometry[0].centroid.coords.xy[0][0], aoi.geometry[0].centroid.coords.xy[1][0])
aoi_utm = aoi.to_crs(epsg_utm)

ee.Initialize()

# Landsat
dataset = 'Landsat'
query_gee_for_imagery_run_pipeline(dataset, aoi_utm, "2019-01-01", "2019-12-01", 5, 11, rgi_id, 
                                   mask_clouds=True, cloud_cover_max=70, aoi_coverage=70, im_out_path=out_path,
                                   verbose=False, im_download=True)

# Sentinel-2 SR
dataset = 'Sentinel-2_SR'
query_gee_for_imagery_run_pipeline(dataset, aoi_utm, "2019-01-01", "2019-12-01", 5, 11, rgi_id, 
                                   mask_clouds=True, cloud_cover_max=70, aoi_coverage=70, im_out_path=out_path,
                                   verbose=False, im_download=True)

# Sentinel-2 SR
dataset = 'Sentinel-2_TOA'
query_gee_for_imagery_run_pipeline(dataset, aoi_utm, "2019-01-01", "2019-12-01", 5, 11, rgi_id, 
                                   mask_clouds=True, cloud_cover_max=70, aoi_coverage=70, im_out_path=out_path,
                                   verbose=False, im_download=True)

# Remove images with < 70% coverage of AOI
fns = sorted(glob.glob(os.path.join(out_path, '*.tif')))
fn_remove_list = []
for fn in tqdm(fns):
    im = rxr.open_rasterio(fn).squeeze()
    im = im.rio.reproject(epsg_utm)
    im = im.isel(band=0)
    mask = geometry_mask(aoi_utm.geometry, transform=im.rio.transform(), invert=True, out_shape=im.shape)
    im_masked = xr.where(mask==1, im, np.nan)
    im_masked = xr.where((im_masked==-9999) | (im_masked==-32768), np.nan, im_masked)
    nreal = im_masked.notnull().sum().item()
    npx = np.count_nonzero(mask)
    percentage_real = (nreal / npx) * 100
    if percentage_real < 70:
        fn_remove_list.append(fn)
# for fn in fn_remove_list:
#     os.remove(fn)

In [ ]:
# Iterate over classified images
for im_classified_fn in tqdm(im_classified_fns):
    # Load classified image
    im_classified = rxr.open_rasterio(im_classified_fn, decode_times=False).squeeze()
    im_classified = xr.where(im_classified==-9999, np.nan, im_classified)
    im_classified = xr.where(im_classified==2, 1, im_classified)
    date = pd.Timestamp(datetime.datetime.strptime(os.path.basename(im_classified_fn).split('_')[0], "%Y%m%dT%H%M%S"))
    source = os.path.basename(im_classified_fn).split(rgi_id + '_')[1].split('_classified.nc')[0]

    # Load multispec image
    try:
        im_fn = glob.glob(os.path.join(out_path, f"{source}_{str(date)[0:10].replace('-', '')}*.tif"))[0]
    except:
        continue
    im = rxr.open_rasterio(im_fn).squeeze()
    if source=='Landsat':
        image_scaler = 36363.63636363636
    else:
        image_scaler = 1e4
    im = xr.where((im==-9999) | (im==-32768), np.nan, im / image_scaler)
    
    # Plot
    fig = plt.figure(figsize=(8,8))
    gs = matplotlib.gridspec.GridSpec(2, 2, figure=fig, height_ratios=[2,1])
    ax = [fig.add_subplot(gs[0,0]), fig.add_subplot(gs[0,1]), fig.add_subplot(gs[1,:])]
    # RGB image
    ax[0].imshow(np.dstack([im.isel(band=2).data, im.isel(band=1).data, im.isel(band=0).data]),
                 extent=(np.min(im.x.data), np.max(im.x.data), np.min(im.y.data), np.max(im.y.data)))
    aoi.plot(ax=ax[0], facecolor='None', edgecolor='k')
    # classified image
    xmin, xmax = -145.3555, -145.045
    ymin, ymax = 63.144, 63.325
    ax[1].imshow(im_classified.data, cmap=cmap, clim=(1,5), 
                 extent=(xmin, xmax, ymax, ymin))
    ax[1].invert_yaxis()
    aoi.plot(ax=ax[1], facecolor='None', edgecolor='k')
    # dummy points for legend
    ax[1].plot(0, 0, 's', color=colors[0], markersize=12, label='Snow')
    ax[1].plot(0, 0, 's', color=colors[2], markersize=12, label='Ice')
    ax[1].plot(0, 0, 's', color=colors[3], markersize=12, label='Rock')
    ax[1].plot(0, 0, 's', color=colors[4], markersize=12, label='Water')
    ax[1].legend(loc='lower right', frameon=False)
    for axis in ax[0:2]:
        axis.set_xlim(xmin, xmax)
        axis.set_ylim(ymin, ymax)
        axis.set_xticks(axis.get_xticks()[1::2])
        axis.set_yticks(axis.get_yticks()[1::2])

    # AAR time series
    ax[2].plot(scs['datetime'], scs['AAR'], '.k')
    scs_date = scs.loc[(scs['datetime']==date) & (scs['source']==source)]
    ax[2].plot(scs_date['datetime'], scs_date['AAR'], '*m', markersize=15)
    ax[2].set_ylim(0,1.05)
    ax[2].grid(True)
    ax[2].set_ylabel('Snow area ratio')
    fig.suptitle(f"{date}\n{source.replace('_', ' ')}")
    
    fig.tight_layout()

    fig_fn = os.path.join(out_path, f"{date}_{rgi_id}_snow_cover.png")
    fig.savefig(fig_fn, dpi=300, bbox_inches='tight')
    plt.close()
        
    

## Model SLA animation

In [ ]:
import glob
plt.rcParams.update({'font.size':12, 'font.sans-serif': 'Arial'})

# Define output directory
out_path = os.path.join(figures_out_path, 'model_SMB_to_SLA_animation')
if not os.path.exists(out_path):
    os.mkdir(out_path)
    print('Made directory for outputs:', out_path)
    
# Grab modeled SMB file names
rgi_id = '1.00032'
bin_fn = sorted(glob.glob(os.path.join(scm_path, 'Rounce_et_al_2023', 'binned', f'{rgi_id}*.nc')))[0]

# Load binned data
bin = xr.open_dataset(bin_fn)
# grab data variables
h = bin.bin_surface_h_initial.data[0] # surface elevation [m]
x = bin.bin_distance.data[0]
b_sum = np.zeros((len(bin.time.data), len(h))) # cumulative SMB
times = [np.datetime64(x) for x in bin.time.data] # datetimes
months = list(pd.DatetimeIndex(times).month) # months of each datetime
# iterate over each time period after 2013
times = [time for time in times if time >= np.datetime64('2012-10-01')]
elas = np.nan*np.zeros(len(times)) # initialize transient ELAs
for j, time in enumerate(tqdm(times)):
    # subset binned data to time
    bin_time = bin.isel(time=j)
    # grab the SMB 
    b_sum[j,:] = bin_time.bin_massbalclim_monthly.data[0]
    # add the previous SMB (restart the count in October)
    if months[j] != 10: 
        b_sum[j,:] += b_sum[j-1,:]
    # If all SMB > 0, ELA = minimum elevation
    if all(b_sum[j,:] > 0):
        elas[j] = np.min(h)
    # If SMB is > 0 and < 0 in some places, linearly interpolate ELA
    elif any(b_sum[j,:] < 0) & any(b_sum[j,:] > 0):
        elas[j] = np.interp(0, np.flip(b_sum[j,:]), np.flip(h))
    # If SMB < 0 everywhere, fit a piecewise linear fit and extrapolate for SMB=0
    elif all(b_sum[j,:] < 0):
        X, y = b_sum[j,:], h
        elas[j] = np.nanmax(h)
    else:
        print('issue')
        
    # Plot results
    if time >= np.datetime64('2013-01-01'):
        fig, ax = plt.subplots(2, 1, gridspec_kw=dict(height_ratios=[3,1]), figsize=(6,6))
        # surface profile
        ax[0].fill_between(x, np.zeros(len(x)), h, color='gray', edgecolor='k', alpha=0.5)
        positive_mask = b_sum[j,:] > 0
        negative_mask = b_sum[j,:] < 0
        # positive mass balance bars
        ax[0].bar(x[positive_mask], b_sum[j,positive_mask]*50, width=126, 
            bottom=h[positive_mask], color='blue', alpha=0.6, label='Positive Mass Balance')
        # negative mass balance bars
        ax[0].bar(x[negative_mask], b_sum[j,negative_mask]*50, width=126, 
                bottom=h[negative_mask], color='red', alpha=0.6, label='Negative Mass Balance')
        # SLA
        ax[0].axhline(elas[j], 0, np.nanmax(x), color='k')
        ax[0].text(7e3, elas[j]+50, 'Snowline altitude', color='k', ha='right')
        ax[0].set_ylim(1.25e3, 3.3e3)
        ax[0].set_yticks([1500, 2000, 2500, 3000])
        ax[0].set_ylabel('Elevation [m]')
        ax[0].set_xlim(0, np.nanmax(x))
        ax[0].set_xticks(np.arange(0, 7.1e3, step=1e3))
        ax[0].set_xticklabels(np.divide(ax[0].get_xticks(), 1e3).astype(int).astype(str))
        ax[0].set_xlabel('km')
        ax[0].spines[['top', 'right']].set_visible(False)
        ax[0].set_title(str(time)[0:7])
        # SLA time series
        ax[1].plot(times, elas, '.k', markersize=5)
        ax[1].plot(times[j], elas[j], '*m', markersize=10)
        ax[1].set_xlim(np.datetime64('2013-01-01'), np.datetime64('2023-01-01'))
        ax[1].set_ylim(1330, 2600)
        ax[1].set_ylabel('Snowline altitude [m]')
        fig.tight_layout()
        # Save figure
        fig_fn = os.path.join(out_path, f"{str(time)[0:7]}.png")
        fig.savefig(fig_fn, dpi=300, bbox_inches='tight')
        plt.close()
        

## ELA sensitivity to air temperature

In [ ]:
# -----Load glacier boundaries
aois_fn = os.path.join(scm_path, 'analysis', 'all_aois.shp')
aois = gpd.read_file(aois_fn)
print('Glacier boundaries loaded')

# -----Load hypsometric indexes
his_fn = os.path.join(scm_path, 'analysis', 'hypsometric_indexes.csv')
his = pd.read_csv(his_fn)
print('Hypsometric indexes loaded')

# -----Load climate clusters
mean_climate_fn = os.path.join(scm_path, 'analysis', 'climate_clusters.csv')
mean_climate = pd.read_csv(mean_climate_fn)
if 'site_name' in list(mean_climate.columns):
    mean_climate.rename(columns={'site_name': 'RGIId'}, inplace=True)
print('Mean weather conditions loaded')

# -----Load RGI O2 Regions
rgi_O2_fn = os.path.join(scm_path, '..', 'GIS_data', 'RGI', 'RGIv7_02Regions', 
                                'RGI2000-v7.0-o2regions-Alaska-westernCanadaUS_clipped_to_country_outlines.shp')
rgi_O2 = gpd.read_file(rgi_O2_fn)
# remove Brooks Range
rgi_O2 = rgi_O2.loc[rgi_O2['o2region']!='01-01']
# add subregion name and color column
rgi_O2[['Subregion', 'color']] = '', ''
for i, o1o2 in enumerate(rgi_O2['o2region'].drop_duplicates().values):
    o1 = int(o1o2[0:2])
    o2 = int(o1o2[3:])
    subregion_name, color = f.determine_subregion_name_color(o1, o2)
    rgi_O2.loc[rgi_O2['o2region']==o1o2, 'Subregion'] = subregion_name
print('RGI O2 regions loaded')

# -----Load GTOPO30
gtopo_fn = '/Users/raineyaberle/Research/PhD/GIS_data/GTOPO30_clip.tif'
gtopo = rxr.open_rasterio(gtopo_fn)
gtopo = xr.where(gtopo==-32768, np.nan, gtopo)
print('GTOPO30 loaded')

# -----Load remotely-sensed ELA coefficients
fits_obs_fn = os.path.join(scm_path, 'analysis', 'linear_fit_observed_monthly_ela_pdd_snowfall.csv')
fits_obs_df = pd.read_csv(fits_obs_fn)
fits_obs_df[['CenLon', 'CenLat', 'Area', 'Zrange', 'Zmin', 'clustName']] = 0, 0, 0, 0, 0, ''
for rgi_id in fits_obs_df['RGIId'].drop_duplicates().values:
    cen_lon, cen_lat, area, zmin, zmax = aois.loc[aois['RGIId']==rgi_id, ['CenLon', 'CenLat', 'Area', 'Zmin', 'Zmax']].values[0]
    clustName = mean_climate.loc[mean_climate['RGIId']==rgi_id, 'clustName'].values[0]
    zrange = zmax-zmin
    fits_obs_df.loc[fits_obs_df['RGIId']==rgi_id, ['CenLon', 'CenLat', 'Area', 'Zrange', 'Zmin', 'clustName']] = cen_lon, cen_lat, area, zrange, zmin, clustName
    
# Sort by cluster order for plotting
fits_obs_df['clustName'] = pd.Categorical(fits_obs_df['clustName'], cluster_order)
fits_obs_df.sort_values(by='clustName', inplace=True)
fits_obs_df.reset_index(drop=True, inplace=True)
print('Remotely-sensed ELA coefficients loaded') 

In [ ]:
fits_obs_df['Zrange_km'] = fits_obs_df['Zrange'] / 1e3

# -----Plot
fontsize=12
plt.rcParams.update({'font.size':fontsize, 'font.sans-serif':'Arial'})
fig = plt.figure(figsize=(10,14))
gs1 = matplotlib.gridspec.GridSpec(2, 2, height_ratios=[1.7,1], width_ratios=[8,1])
gs2 = matplotlib.gridspec.GridSpec(2, 2, height_ratios=[1.7,1])
ax = [fig.add_axes([0.0, 0.3, 0.6, 0.6]), 
      fig.add_axes([0.66, 0.623, 0.27, 0.15]),
      fig.add_subplot(gs2[1,:])]#, fig.add_subplot(gs2[1,1])]

### a) Map view
# GTOPO hillshade
ls = matplotlib.colors.LightSource(azdeg=90, altdeg=45)
ax[0].imshow(ls.hillshade(gtopo.data[0], vert_exag=0.002), cmap='gray', alpha=0.5,
             extent=(np.min(gtopo.x.data), np.max(gtopo.x.data), 
                     np.min(gtopo.y.data), np.max(gtopo.y.data)))
# RGI O2 region outlines
color = '#525252'
rgi_O2.plot(ax=ax[0], alpha=1.0, facecolor='None', edgecolor=color, linewidth=1)
ax[0].set_yticks(np.linspace(45, 65, num=6))
ax[0].set_xlim(-167, -112)
ax[0].set_ylim(46, 67)
ax[0].set_xlabel('Longitude')
ax[0].set_ylabel('Latitude')
ax[0].set_aspect(2.1)
# PDD coefficients
sns.scatterplot(data=fits_obs_df, x='CenLon', y='CenLat', edgecolor='k', linewidth=0.5, 
                hue='clustName', palette=cluster_cmap_dict, alpha=1, size='Zrange_km', 
                sizes=(5,100), ax=ax[0])
handles, labels = ax[0].get_legend_handles_labels()
labels = [x.replace('clustName', 'Weather class').replace('coef_PDD_median', '$\Sigma$PDD coefficient [m/$^{\circ}$C]').replace('Zrange_km', 'Elevation range [km]')
          for x in labels]
ax[0].legend().remove()
fig.legend(handles, labels, loc='center right', bbox_to_anchor=[0.7, 0.38, 0.2, 0.2], fontsize=fontsize-1)
# Add region labels and arrows
fontweight = 'bold'
ax[0].text(-163, 56, 'Aleutians', color=color, rotation=35, fontsize=fontsize-3, fontweight=fontweight)
ax[0].text(-158, 62.3, 'Alaska Range', color=color, rotation=0, fontsize=fontsize-3, fontweight=fontweight)
ax[0].text(-147.9, 57.8, 'W. Chugach \nMtns.', color=color, rotation=0, horizontalalignment='center', fontsize=fontsize-3, fontweight=fontweight)
ax[0].arrow(-147.6, 58.8, 0, 0.8, color=color, linewidth=2, head_width=0.25, head_length=0.2)
ax[0].text(-141.7, 57.7, 'St. Elias \nMtns.', color=color, rotation=0, horizontalalignment='center', fontsize=fontsize-3, fontweight=fontweight)
ax[0].arrow(-141.5, 58.7, 0, 0.8, color=color, linewidth=2, head_width=0.25, head_length=0.2)
ax[0].text(-139.6, 56.4, 'N. Coast \nRanges', color=color, rotation=0, horizontalalignment='center', fontsize=fontsize-3, fontweight=fontweight)
ax[0].arrow(-137.3, 56.8, 1.3, 0, color=color, linewidth=2, head_width=0.25, head_length=0.2)
ax[0].text(-133, 51.3, 'N. Cascades', color=color, rotation=0, horizontalalignment='center', fontsize=fontsize-3, fontweight=fontweight)
ax[0].arrow(-129.4, 51.4, 1.3, 0, color=color, linewidth=2, head_width=0.25, head_length=0.2)
ax[0].text(-129.7, 47, 'S. Cascades', color=color, rotation=0, horizontalalignment='center', fontsize=fontsize-3, fontweight=fontweight)
ax[0].arrow(-126, 47.1, 1.3, 0, color=color, linewidth=2, head_width=0.25, head_length=0.2)
ax[0].text(-132, 64, 'N. Rockies', color=color, rotation=0, fontsize=fontsize-3, fontweight=fontweight)
ax[0].text(-122, 55, 'C. Rockies', color=color, rotation=0, fontsize=fontsize-3, fontweight=fontweight)
ax[0].text(-117.7, 47, 'S. Rockies', color=color, rotation=0, fontsize=fontsize-3, fontweight=fontweight)

### b) Mean weather conditions
sns.scatterplot(data=mean_climate, x='mean_annual_temp_range', y='mean_annual_precip_cumsum', 
                hue='clustName', palette=cluster_cmap_dict, alpha=1, ax=ax[1], s=15, legend=False)
ax[1].set_xlabel('Air temperature range [$^{\circ}$C]', fontsize=fontsize)
ax[1].set_ylabel('Precipitation sum [m]', fontsize=fontsize)

### c) Scatterplots
sns.scatterplot(fits_obs_df, x='coef_PDD_median', y='Subregion', hue='clustName', size='Zrange_km', edgecolor='k', linewidth=0.5,
                sizes=(5,100), palette=cluster_cmap_dict, hue_order=cluster_order, ax=ax[2])
ax[2].legend().remove()
ax[2].yaxis.set_minor_locator(matplotlib.ticker.FixedLocator(np.arange(-0.5, 9.5, step=1)))
ax[2].tick_params(which='minor', length=0)
ax[2].yaxis.grid(True, which='minor')
ax[2].set_ylabel('')
ax[2].set_xlabel('$\Sigma$PDD coefficient [m/$^{\circ}$C]')
ax[2].set_xlim(0, 7)

# Add text labels
text_labels = ['a', 'b', 'c']
for i, axis in enumerate(ax):
    if i==1:
        scale = 0.85
    else:
        scale = 0.9
    axis.text((axis.get_xlim()[1] - axis.get_xlim()[0])*scale + axis.get_xlim()[0],
              (axis.get_ylim()[1] - axis.get_ylim()[0])*scale + axis.get_ylim()[0],
              text_labels[i], fontsize=fontsize+4, fontweight='bold')

fig.subplots_adjust(wspace=0.4, hspace=0.15)
plt.show()

# Save figure
fig_fn = os.path.join(figures_out_path, 'fig02_ELA_PDD_sensitivities.png')
fig.savefig(fig_fn, dpi=300, bbox_inches='tight')
print('Figure saved to file:', fig_fn)

# Print statistics by subregion and climate cluster

In [ ]:
fits_obs_df.groupby(['Subregion', 'clustName'])['coef_PDD_median'].describe()

In [ ]:
np.nanpercentile(fits_obs_df.loc[(fits_obs_df['Subregion']=='S. Rockies') | (fits_obs_df['Subregion']=='N. Rockies') | (fits_obs_df['Subregion']=='C. Rockies'), 'coef_PDD_median'], 100)

### Filter by regression score to see if trends persist

In [ ]:
fits_obs_filt_df = fits_obs_df.loc[fits_obs_df['score_mean'] > 0.2]

# -----Plot
fontsize=12
plt.rcParams.update({'font.size':fontsize, 'font.sans-serif':'Arial'})
fig = plt.figure(figsize=(10,14))
gs1 = matplotlib.gridspec.GridSpec(2, 2, height_ratios=[1.7,1], width_ratios=[8,1])
gs2 = matplotlib.gridspec.GridSpec(2, 1, height_ratios=[1.7,1])
ax = [fig.add_axes([0.0, 0.3, 0.6, 0.6]), 
      fig.add_axes([0.66, 0.623, 0.27, 0.15]),
      fig.add_subplot(gs2[1,0])]

### a) Map view
# GTOPO hillshade
ls = matplotlib.colors.LightSource(azdeg=90, altdeg=45)
ax[0].imshow(ls.hillshade(gtopo.data[0], vert_exag=0.002), cmap='gray', alpha=0.5,
             extent=(np.min(gtopo.x.data), np.max(gtopo.x.data), 
                     np.min(gtopo.y.data), np.max(gtopo.y.data)))
# RGI O2 region outlines
color = '#525252'
rgi_O2.plot(ax=ax[0], alpha=1.0, facecolor='None', edgecolor=color, linewidth=1)
ax[0].set_yticks(np.linspace(45, 65, num=6))
ax[0].set_xlim(-167, -112)
ax[0].set_ylim(46, 67)
ax[0].set_xlabel('Longitude')
ax[0].set_ylabel('Latitude')
ax[0].set_aspect(2.1)
# PDD coefficients
sns.scatterplot(data=fits_obs_filt_df, x='CenLon', y='CenLat', edgecolor='k', linewidth=0.7, 
                hue='clustName', palette=cluster_cmap_dict, alpha=1, size='Zrange_km', 
                sizes=(5,100), ax=ax[0])
handles, labels = ax[0].get_legend_handles_labels()
labels = [x.replace('clustName', 'Weather class').replace('Zrange_km', 'Elevation range [km]') 
          for x in labels]
ax[0].legend().remove()
# Size legend in axis
# ax[0].legend(handles[6:], labels[6:], loc='lower left', fontsize=fontsize-2, bbox_to_anchor=[0.15, 0.05, 0.2, 0.2])
# Weather class legend outside axis
fig.legend(handles, labels, loc='center right', bbox_to_anchor=[0.7, 0.38, 0.2, 0.2], fontsize=fontsize-1)

# Add region labels and arrows
fontweight = 'bold'
ax[0].text(-163, 56, 'Aleutians', color=color, rotation=35, fontsize=fontsize-3, fontweight=fontweight)
ax[0].text(-158, 62.3, 'Alaska Range', color=color, rotation=0, fontsize=fontsize-3, fontweight=fontweight)
ax[0].text(-147.9, 57.8, 'W. Chugach \nMtns.', color=color, rotation=0, horizontalalignment='center', fontsize=fontsize-3, fontweight=fontweight)
ax[0].arrow(-147.6, 58.8, 0, 0.8, color=color, linewidth=2, head_width=0.25, head_length=0.2)
ax[0].text(-141.7, 57.7, 'St. Elias \nMtns.', color=color, rotation=0, horizontalalignment='center', fontsize=fontsize-3, fontweight=fontweight)
ax[0].arrow(-141.5, 58.7, 0, 0.8, color=color, linewidth=2, head_width=0.25, head_length=0.2)
ax[0].text(-139.6, 56.4, 'N. Coast \nRanges', color=color, rotation=0, horizontalalignment='center', fontsize=fontsize-3, fontweight=fontweight)
ax[0].arrow(-137.3, 56.8, 1.3, 0, color=color, linewidth=2, head_width=0.25, head_length=0.2)
ax[0].text(-133, 51.3, 'N. Cascades', color=color, rotation=0, horizontalalignment='center', fontsize=fontsize-3, fontweight=fontweight)
ax[0].arrow(-129.4, 51.4, 1.3, 0, color=color, linewidth=2, head_width=0.25, head_length=0.2)
ax[0].text(-129.7, 47, 'S. Cascades', color=color, rotation=0, horizontalalignment='center', fontsize=fontsize-3, fontweight=fontweight)
ax[0].arrow(-126, 47.1, 1.3, 0, color=color, linewidth=2, head_width=0.25, head_length=0.2)
ax[0].text(-132, 64, 'N. Rockies', color=color, rotation=0, fontsize=fontsize-3, fontweight=fontweight)
ax[0].text(-122, 55, 'C. Rockies', color=color, rotation=0, fontsize=fontsize-3, fontweight=fontweight)
ax[0].text(-117.7, 47, 'S. Rockies', color=color, rotation=0, fontsize=fontsize-3, fontweight=fontweight)

### b) Mean weather conditions
sns.scatterplot(data=mean_climate, x='mean_annual_temp_range', y='mean_annual_precip_cumsum', 
                hue='clustName', palette=cluster_cmap_dict, alpha=1, ax=ax[1], s=15, legend=False)
ax[1].set_xlabel('Air temperature range [$^{\circ}$C]', fontsize=fontsize)
ax[1].set_ylabel('Precipitation sum [m]', fontsize=fontsize)

### c) and d) Box plots by subregion and climate class
sns.scatterplot(data=fits_obs_filt_df, x='coef_PDD_median', y='Subregion', hue='clustName', size='Zrange_km',
                alpha=0.9, palette=cluster_cmap_dict, sizes=(5,100), legend=False)
ax[2].set_ylabel('')
ax[2].set_xlabel('$\Sigma$PDD coefficient [m/$^{\circ}$C]')
ax[2].grid(axis='x')
ax[2].set_xlim(0,7)
# add minor gridlines on y-axis
ax[2].yaxis.set_minor_locator(matplotlib.ticker.FixedLocator(np.arange(-0.5, 9.5, step=1)))
ax[2].tick_params(which='minor', length=0)
ax[2].yaxis.grid(True, which='minor')

# Add text labels
text_labels = ['a', 'b', 'c']
for i, axis in enumerate(ax):
    if i==1:
        scale = 0.85
    else:
        scale = 0.9
    axis.text((axis.get_xlim()[1] - axis.get_xlim()[0])*scale + axis.get_xlim()[0],
              (axis.get_ylim()[1] - axis.get_ylim()[0])*scale + axis.get_ylim()[0],
              text_labels[i], fontsize=fontsize+4, fontweight='bold')

fig.subplots_adjust(wspace=0.4, hspace=0.15)
plt.show()

# Save figure
fig_fn = os.path.join(figures_out_path, 'fig02_ELA_PDD_sensitivities_R2_gt_0.2.png')
fig.savefig(fig_fn, dpi=300, bbox_inches='tight')
print('Figure saved to file:', fig_fn)

## AAR time series

In [ ]:
# -----Add Year and WOY columns to snowlines
snowlines['Year'] = snowlines['datetime'].dt.isocalendar().year
snowlines['WOY'] = snowlines['datetime'].dt.isocalendar().week

# -----Add subregion and climate cluster columns to snowlines
snowlines[['Subregion', 'color']] = '', '', 
for o1, o2 in tqdm(aois[['O1Region', 'O2Region']].drop_duplicates().values):
    subregion_name, color = f.determine_subregion_name_color(o1, o2)
    aois_subregion = aois.loc[(aois['O1Region']==o1) & (aois['O2Region']==o2)]
    site_names = aois_subregion['RGIId'].drop_duplicates().values
    for site_name in site_names:
        snowlines.loc[snowlines['site_name']==site_name, 'O1Region'] = o1
        snowlines.loc[snowlines['site_name']==site_name, 'O2Region'] = o2
        snowlines.loc[snowlines['site_name']==site_name, 'Subregion'] = subregion_name
        snowlines.loc[snowlines['site_name']==site_name, 'color'] = color
snowlines[['cluster', 'clustName']] = '', '', 
for site_name in (climate_clusters['site_name'].drop_duplicates().values):
    climate_clusters_site = climate_clusters.loc[climate_clusters['site_name']==site_name]
    snowlines.loc[snowlines['site_name']==site_name, 'cluster'] = climate_clusters_site['cluster'].values[0]
    snowlines.loc[snowlines['site_name']==site_name, 'clustName'] = climate_clusters_site['clustName'].values[0]
snowlines = snowlines.loc[snowlines['cluster']!='']
snowlines.reset_index(drop=True, inplace=True)

### By subregion

In [ ]:
# -----Set up figure
fig, ax = plt.subplots(10, 2, figsize=(12, 40), gridspec_kw={'width_ratios':[4,1]})
ax = ax.flatten()

def add_date_column(df):
    df['date'] = pd.to_datetime(df['Year'].astype(str) + df['WOY'].astype(str) + '1', format='%Y%W%w')
    df['date'] = df['date'] - pd.Timedelta(days=1)
    return df

q1, q3 = 0.25, 0.75

# -----Iterate over subregions
o1o2s = snowlines[['O1Region', 'O2Region']].drop_duplicates().dropna().sort_values(by=['O1Region', 'O2Region']).values
ymin, ymax = -0.1, 1.1
for i, (o1, o2) in enumerate(o1o2s):
    # subset snowlines to subregion
    snowlines_subregion = snowlines.loc[(snowlines['O1Region']==o1) & (snowlines['O2Region']==o2)]
    if len(snowlines_subregion) < 1:
        continue
    subregion_name, color = snowlines_subregion[['Subregion', 'color']].values[0]
    if color=='':
        color='k'
    # calculate moving median over time
    ts_gb = snowlines_subregion.groupby(by=['Year', 'WOY'])['AAR']
    ts = ts_gb.median().reset_index()
    ts = add_date_column(ts)
    ts.rename(columns={'AAR': 'AAR_median'}, inplace=True)
    ts['AAR_Q1'] = ts_gb.quantile(q1).reset_index()['AAR']
    ts['AAR_Q3'] = ts_gb.quantile(q3).reset_index()['AAR']
    # calculate moving median for all years stacked
    weekly_gb = snowlines_subregion.groupby(by='WOY')['AAR']
    weekly = weekly_gb.median().reset_index()
    weekly.rename(columns={'AAR': 'AAR_median'}, inplace=True)
    weekly['AAR_Q1'] = weekly_gb.quantile(q1).reset_index()['AAR']
    weekly['AAR_Q3'] = weekly_gb.quantile(q3).reset_index()['AAR']
    weekly['WOY'] = weekly['WOY'].values.astype(float)
    # plot
    # add grey boxes where observations don't exist
    for year in ts['Year'].values:
        t1, t2 = pd.to_datetime(str(year-1)+'-11-01'), pd.to_datetime(str(year)+'-05-01')
        ax[i*2].add_patch(Rectangle((t1, ymin), width=t2-t1, height=ymax-ymin, facecolor='#d9d9d9', edgecolor='None'))
        ts_year = ts.loc[ts['Year']==year]
        ax[i*2].plot(ts_year['date'], ts_year['AAR_median'], 'o', color=color, linewidth=0.5, 
                     markerfacecolor=color, markeredgecolor='w', markersize=5)
    ax[i*2].set_ylim(ymin, ymax)
    ax[i*2].grid()
    ax[i*2].set_ylabel('AAR')
    N = len(snowlines_subregion['site_name'].drop_duplicates())
    ax[i*2].set_title(f'{subregion_name} (N={N})')
    ax[i*2].xaxis.set_major_formatter(matplotlib.dates.DateFormatter("%Y"))
    ax[(i*2)+1].set_ylim(ymin, ymax)
    ax[(i*2)+1].grid()
    ax[(i*2)+1].fill_between(weekly['WOY'], weekly['AAR_Q1'], weekly['AAR_Q3'], color=color, alpha=0.5)
    ax[(i*2)+1].plot(weekly['WOY'], weekly['AAR_median'], '-', linewidth=2, color=color)
    # change week numbers to month labels
    ax[(i*2)+1].set_xticks([22, 31, 40])
    ax[(i*2)+1].set_xticklabels(['June', 'Aug', 'Oct'])

fig.subplots_adjust(hspace=0.3)
plt.show()

# -----Save figure
fig_fn = os.path.join(figures_out_path, 'timeseries_aar_subregions.png')
fig.savefig(fig_fn, dpi=250, bbox_inches='tight')
print('figure saved to file:', fig_fn)

### By climate cluster

In [ ]:
# -----Set up figure
n_clusters = len(snowlines['cluster'].drop_duplicates().values)
fig, ax = plt.subplots(n_clusters, 2, figsize=(12, 4*n_clusters), gridspec_kw={'width_ratios':[4,1]})
ax = ax.flatten()

def add_date_column(df):
    df['date'] = pd.to_datetime(df['Year'].astype(str) + df['WOY'].astype(str) + '1', format='%Y%W%w')
    df['date'] = df['date'] - pd.Timedelta(days=1)
    return df

q1, q3 = 0.25, 0.75

# -----Iterate over subregions
ymin, ymax = -0.1, 1.1
for i, cluster_name in enumerate(['Oceanic', 
                                  'Continental', 
                                  'Transitional-Continental', 
                                  'Transitional-Temperate', 
                                  'Temperate']):
    # subset snowlines to cluster
    snowlines_cluster = snowlines.loc[snowlines['clustName']==cluster_name]
    if len(snowlines_cluster) < 1:
        continue
    # grab cluster name
    color = cluster_cmap_dict[cluster_name]
    # calculate moving median over time
    ts_gb = snowlines_cluster.groupby(by=['Year', 'WOY'])['AAR']
    ts = ts_gb.median().reset_index()
    ts = add_date_column(ts)
    ts.rename(columns={'AAR': 'AAR_median'}, inplace=True)
    ts['AAR_Q1'] = ts_gb.quantile(q1).reset_index()['AAR']
    ts['AAR_Q3'] = ts_gb.quantile(q3).reset_index()['AAR']
    # calculate moving median for all years stacked
    weekly_gb = snowlines_cluster.groupby(by='WOY')['AAR']
    weekly = weekly_gb.median().reset_index()
    weekly.rename(columns={'AAR': 'AAR_median'}, inplace=True)
    weekly['AAR_Q1'] = weekly_gb.quantile(q1).reset_index()['AAR']
    weekly['AAR_Q3'] = weekly_gb.quantile(q3).reset_index()['AAR']
    weekly['WOY'] = weekly['WOY'].values.astype(float)
    # plot
    # add grey boxes where observations don't exist
    for year in ts['Year'].values:
        t1, t2 = pd.to_datetime(str(year-1)+'-11-01'), pd.to_datetime(str(year)+'-05-01')
        ax[i*2].add_patch(Rectangle((t1, ymin), width=t2-t1, height=ymax-ymin, facecolor='#d9d9d9', edgecolor='None'))
        ts_year = ts.loc[ts['Year']==year]
        ax[i*2].plot(ts_year['date'], ts_year['AAR_median'], 'o', color=color, linewidth=0.5, 
                     markerfacecolor=color, markeredgecolor='w', markersize=5)
    ax[i*2].set_ylim(ymin, ymax)
    ax[i*2].grid()
    ax[i*2].set_ylabel('AAR')
    N = len(snowlines_cluster['site_name'].drop_duplicates())
    ax[i*2].set_title(f'{cluster_name} (N={N})')
    ax[i*2].xaxis.set_major_formatter(matplotlib.dates.DateFormatter("%Y"))
    ax[(i*2)+1].set_ylim(ymin, ymax)
    ax[(i*2)+1].grid()
    ax[(i*2)+1].fill_between(weekly['WOY'], weekly['AAR_Q1'], weekly['AAR_Q3'], color=color, alpha=0.5)
    ax[(i*2)+1].plot(weekly['WOY'], weekly['AAR_median'], '-', linewidth=2, color=color)
    # change week numbers to month labels
    ax[(i*2)+1].set_xticks([22, 31, 40])
    ax[(i*2)+1].set_xticklabels(['June', 'Aug', 'Oct'])

fig.subplots_adjust(hspace=0.3)
plt.show()

# -----Save figure
fig_fn = os.path.join(figures_out_path, 'timeseries_aar_clusters.png')
fig.savefig(fig_fn, dpi=250, bbox_inches='tight')
print('figure saved to file:', fig_fn)

### Annual trends stacked

In [ ]:
# -----Set up figure
fig, ax = plt.subplots(2, 5, figsize=(14, 8))
ax = ax.flatten()

def add_date_column(df):
    df['date'] = pd.to_datetime(df['Year'].astype(str) + df['WOY'].astype(str) + '1', format='%Y%W%w')
    df['date'] = df['date'] - pd.Timedelta(days=1)
    return df

cmap = plt.cm.viridis
    
# -----Iterate over subregions
o1o2s = snowlines[['O1Region', 'O2Region']].drop_duplicates().dropna().sort_values(by=['O1Region', 'O2Region']).values
ymin, ymax = -0.1, 1.1
for i, (o1, o2) in enumerate(o1o2s):
    # subset snowlines to subregion
    snowlines_subregion = snowlines.loc[(snowlines['O1Region']==o1) & (snowlines['O2Region']==o2)]
    if len(snowlines_subregion) < 1:
        continue
    subregion_name, color = snowlines_subregion[['Subregion', 'color']].values[0]
    if color=='':
        color='k'
    # calculate moving median for all years separately
    yearly_gb = snowlines_subregion.groupby(by=['Year', 'WOY'])['AAR']
    yearly = yearly_gb.median().reset_index()
    yearly.rename(columns={'AAR': 'AAR_median'}, inplace=True)
    # plot
    for year in yearly['Year'].drop_duplicates().values:
        yearly_year = yearly.loc[yearly['Year']==year]
        ax[i].plot(yearly_year['WOY'], yearly_year['AAR_median'], '-', color=cmap((year-2013)/(2023-2013)), linewidth=1)
    ax[i].set_ylim(ymin, ymax)
    ax[i].grid()
    if (i==0) or (i==5):
        ax[i].set_ylabel('AAR')
    else:
        ax[i].set_yticklabels([])
    if i < 5:
        ax[i].set_xticklabels([])
    N = len(snowlines_subregion['site_name'].drop_duplicates())
    ax[i].set_title(f'{subregion_name} (N={N})')

cbar_ax = fig.add_axes([0.94, 0.4, 0.01, 0.3])
fig.colorbar(matplotlib.cm.ScalarMappable(norm=matplotlib.colors.Normalize(2013, 2023), cmap=cmap),
             cax=cbar_ax, orientation='vertical')
fig.subplots_adjust(hspace=0.2)
plt.show()

# -----Save figure
fig_fn = os.path.join(figures_out_path, 'timeseries_aar_years_stacked.png')
fig.savefig(fig_fn, dpi=250, bbox_inches='tight')
print('figure saved to file:', fig_fn)

### Weather data

In [ ]:
# -----Load ERA data
era_fn = os.path.join(scm_path, 'all_ERA_data', 'all_era_data.csv')
era = pd.read_csv(era_fn)
era['Date'] = pd.to_datetime(era['Date'])
# Add Year and WOY columns
era['Year'] = era['Date'].dt.isocalendar().year
era['WOY'] = era['Date'].dt.isocalendar().week

# -----Load AOIs
aois = gpd.read_file(os.path.join(aois_path, aois_fn))
aois[['O1Region', 'O2Region']] = aois[['O1Region', 'O2Region']].astype(int)

# -----Add o1 and o2 region columns
era[['O1Region', 'O2Region']] = '', ''
for site_name in tqdm(era['site_name'].drop_duplicates().values):
    aoi = aois.loc[aois['RGIId']==site_name]
    o1, o2 = aoi[['O1Region', 'O2Region']].values[0]
    era.loc[era['site_name']==site_name, 'O1Region'] = o1
    era.loc[era['site_name']==site_name, 'O2Region'] = o2
era

In [ ]:
# -----Set up figure
plt.rcParams.update({'font.sans-serif':'Arial', 'font.size':12})
fig, ax = plt.subplots(10, 2, figsize=(14, 35), gridspec_kw={'width_ratios':[4,1]})
ax = ax.flatten()
ymin1, ymax1 = 0, 1.05
ymin2, ymax2 = 0, 1600
ymin3, ymax3 = 0, 4

weather_columns = ['Cumulative_Positive_Degree_Days', 'Cumulative_Snowfall_mwe']
colors = ['#f4a582', '#2166ac']
q1, q3 = 0.25, 0.75

def add_date_column(df):
    df['date'] = pd.to_datetime(df['Year'].astype(str) + df['WOY'].astype(str) + '1', format='%Y%W%w')
    df['date'] = df['date'] - pd.Timedelta(days=1)
    return df

def calculate_weekly_trends(df, column):
    ts_gb = df.groupby(by=['Year', 'WOY'])[column]
    ts = ts_gb.median().reset_index()
    ts = add_date_column(ts)
    ts.rename(columns={column: column+'_median'}, inplace=True)
    ts[column+'_Q1'] = ts_gb.quantile(q1).reset_index()[column]
    ts[column+'_Q3'] = ts_gb.quantile(q3).reset_index()[column]
    # calculate moving median for all years stacked
    weekly_gb = df.groupby(by='WOY')[column]
    weekly = weekly_gb.median().reset_index()
    weekly.rename(columns={column: column+'_median'}, inplace=True)
    weekly[column+'_Q1'] = weekly_gb.quantile(q1).reset_index()[column]
    weekly[column+'_Q3'] = weekly_gb.quantile(q3).reset_index()[column]

    return ts, weekly    

# -----Iterate over subregions
for i, (o1, o2) in enumerate(era[['O1Region', 'O2Region']].drop_duplicates().values):
    
    subregion_name, color = f.determine_subregion_name_color(o1, o2)
    print(subregion_name)
    
    # subset snowlines and ERA to subregion
    era_subregion = era.loc[(era['O1Region']==o1) & (era['O2Region']==o2)]
    snowlines_subregion = snowlines.loc[(snowlines['O1Region']==o1) & (snowlines['O2Region']==o2)]

    # calculate moving medians over time and plot
    # PDDs
    column, color = weather_columns[0], colors[0]
    ts, weekly = calculate_weekly_trends(era_subregion, column)
    for year in ts['Year'].drop_duplicates().values + 1:
        t1, t2 = pd.to_datetime(str(year-1)+'-11-01'), pd.to_datetime(str(year)+'-05-01')
        ax[i*2].add_patch(Rectangle((t1, ymin1), width=t2-t1, height=ymax1-ymin1, 
                                    facecolor='#d9d9d9', edgecolor='None', zorder=1))
    twin1 = ax[i*2].twinx()
    twin1.fill_between(ts['date'], ts[column+'_median'], np.zeros(len(ts)), 
                       facecolor=color, alpha=0.5, zorder=2)
    twin1.set_ylabel('$\Sigma$PDDs', color=color)
    twin1.spines['left'].set_color(color)
    twin1.tick_params(axis='y', colors=color)
    twin1.set_ylim(ymin2, ymax2)
    # Snowfall
    twin2 = ax[i*2].twinx()
    column, color = weather_columns[1], colors[1]
    ts, weekly = calculate_weekly_trends(era_subregion, column)
    twin2.plot(ts['date'], ts[column+'_median'], '-', color=color, linewidth=2, zorder=3)
    twin2.set_ylabel('$\Sigma$Snowfall [m.w.e.]', color=color)
    twin2.spines.right.set_color(color)
    twin2.spines.right.set_position(("axes", 1.1))
    twin2.tick_params(axis='y', colors=color)
    twin2.set_ylim(ymin3, ymax3)
    # AAR
    column, color = 'AAR', 'k'
    ts, weekly = calculate_weekly_trends(snowlines_subregion, column)
    ax[i*2].set_ylim(ymin1, ymax1)
    ax[i*2].plot(ts['date'], ts[column+'_median'], '.', color=color, markersize=7, zorder=5)
    ax[i*2].set_ylabel('AAR', color=color)
    ax[(i*2)+1].plot(weekly['WOY'], weekly[column+'_Q1'], '-', linewidth=1, color=color, alpha=0.5)
    ax[(i*2)+1].plot(weekly['WOY'], weekly[column+'_Q3'], '-', linewidth=1, color=color, alpha=0.5)
    ax[(i*2)+1].plot(weekly['WOY'], weekly[column+'_median'], '-', linewidth=2, color=color)
    ax[(i*2)+1].set_ylim(ymin1, ymax1)
    # adjust axes
    N = len(era_subregion['site_name'].drop_duplicates())
    ax[i*2].set_title(f'{subregion_name} (N={N})')
    ax[i*2].xaxis.set_major_formatter(matplotlib.dates.DateFormatter("%Y"))
    ax[i*2].set_xlim(np.datetime64('2013-01-01'), np.datetime64('2024-02-01'))
    ax[(i*2)+1].grid()

fig.subplots_adjust(hspace=0.25, wspace=0.4)
plt.show()

# -----Save figure
fig_fn = os.path.join(figures_out_path, 'timeseries_aar_pdd_snowfall.png')
fig.savefig(fig_fn, dpi=250, bbox_inches='tight')
print('figure saved to file:', fig_fn)